# ITR Data Pipeline

The ITR data pipeline organizes and assembles data needed for the ITR tool.  The data may come from many sources, but the output of this pipeline is a complete, consistent dataset that can be fully interrogated by the ITR tool.  If users wish to add additional data or analyze additional portfolio companies, they must create a new dataset using this pipeline.

These are the data needed to create the ITR dataset:
* Global Parameters (just for reference--we do nothing with them here)
* Industry Data (Sector Projections aka Benchmarks)
* Portfolio Data (Must cover all the stocks a user may query)
* Company Data (Must cover all companies in all possible portfolio universes)
* Automization (Must cover all years and scenarios a user may query)

The ITR tool can create secondary datasets:
* Cumulative emissions targets trajectories
* Cumulative emissions budgets
* Target and trajectory overshoot/undershoot ratios
* Target and trajectory temperature scores

These secondary datasets are not the concern of this pipeline.

### Environment variables and dot-env

The following cell looks for a "dot-env" file in some standard locations,
and loads its contents into `os.environ`.

In [1]:
import os
import pathlib
from dotenv import load_dotenv

# Load some standard environment variables from a dot-env file, if it exists.
# If no such file can be found, does not fail, and so allows these environment vars to
# be populated in some other way
dotenv_dir = os.environ.get('CREDENTIAL_DOTENV_DIR', os.environ.get('PWD', '/opt/app-root/src'))
dotenv_path = pathlib.Path(dotenv_dir) / 'credentials.env'
if os.path.exists(dotenv_path):
    load_dotenv(dotenv_path=dotenv_path,override=True)

import numpy as np
import pandas as pd

### S3 and boto3

In [2]:
import boto3

s3_source = boto3.resource(
    service_name="s3",
    endpoint_url=os.environ['S3_LANDING_ENDPOINT'],
    aws_access_key_id=os.environ['S3_LANDING_ACCESS_KEY'],
    aws_secret_access_key=os.environ['S3_LANDING_SECRET_KEY'],
)
source_bucket = s3_source.Bucket(os.environ['S3_LANDING_BUCKET'])

In [3]:
import osc_ingest_trino as osc
from ITR.data.osc_units import *
import io

### Connecting to Trino with sqlalchemy

In the context of the Data Vault, this pipeline operates with full visibiilty into all the data it prepares for the ITR tool.  When the data is output, it is labeled so that the Data Vault can enforce its data management access rules.

In [4]:
import trino
from sqlalchemy.engine import create_engine

ingest_catalog = 'osc_datacommons_dev'
ingest_schema = 'sandbox'
dera_schema = 'sandbox'
dera_prefix = 'dera_'
gleif_schema = 'sandbox'
rmi_schema = 'sandbox'
iso3166_schema = 'sandbox'
essd_schema = 'sandbox'
demo_schema = 'demo_dv'

sqlstring = 'trino://{user}@{host}:{port}/'.format(
    user = os.environ['TRINO_USER'],
    host = os.environ['TRINO_HOST'],
    port = os.environ['TRINO_PORT']
)
sqlargs = {
    'auth': trino.auth.JWTAuthentication(os.environ['TRINO_PASSWD']),
    'http_scheme': 'https',
    'catalog': ingest_catalog,
    'schema': ingest_schema,
}
engine = create_engine(sqlstring, connect_args = sqlargs)
connection = engine.connect()

/Users/michael/opt/miniconda3/envs/jupyter/lib/python3.10/site-packages/sqlalchemy_trino/dialect.py:259: SAWarning: Dialect trino:rest will not make use of SQL compilation caching as it does not set the 'supports_statement_cache' attribute to ``True``.  This can have significant performance implications including some performance degradations in comparison to prior SQLAlchemy versions.  Dialect maintainers should seek to set this attribute to True after appropriate development and testing for SQLAlchemy 1.4 caching support.   Alternatively, this attribute may be set to False which will disable this warning. (Background on this error at: https://sqlalche.me/e/14/cprf)
  res = connection.execute(sql.text(query)).scalar()


## Global Parameters

These parameters are set/selected by the ITR tool.  They are included here for reference only (the following is not live code).

Create the ISIC-to-Sector table manually until we have a proper sector mapping table

In [5]:
i2s_df = pd.DataFrame({"isic": [2410, 4010],
                       "sector": ['Steel', 'Electricity Utilities']}).convert_dtypes()

ingest_table = 'isic_to_sector'
drop_table = engine.execute(f"drop table if exists {demo_schema}.{ingest_table}")
drop_table.fetchall()

columnschema = osc.create_table_schema_pairs(i2s_df)

tabledef = f"""
create table if not exists {ingest_catalog}.{demo_schema}.{ingest_table}(
{columnschema}
) with (
    format = 'ORC',
    partitioning = array['bucket(isic,20)']
)
"""

print(tabledef)
qres = engine.execute(tabledef)
print(qres.fetchall())
i2s_df.to_sql(ingest_table,
              con=engine, schema=demo_schema, if_exists='append',
              index=False,
              method=osc.TrinoBatchInsert(batch_size = 2000, verbose = True))


create table if not exists osc_datacommons_dev.demo_dv.isic_to_sector(
    isic bigint,
    sector varchar
) with (
    format = 'ORC',
    partitioning = array['bucket(isic,20)']
)

[(True,)]
constructed fully qualified table name as: "demo_dv.isic_to_sector"
inserting 2 records
  (2410, 'Steel')
  (4010, 'Electricity Utilities')
batch insert result: [(2,)]


## Portfolio Data

The user will ultimately supply portfolio selection and position information to the ITR tool as part of the weighting calculations.  This part of the pipeline just collects the LEI and ISIN information for companies we should expect to analyze (i.e., companies for which we have fundamental financial information, production, intensity, and target information, in sectors for which we have benchmark projections).

Because this pipeline does the full pre-computation of data for the tool, there is no sense carrying forward information that is not fully closed.  I.e., there's no reason to carry forward an LEI:ISIN relationship if there is no financial, production, or target information related to that LEI and/or ISIN.  The user does not add such data later; the data is collected and fully processed by this pipeline now.

### Get LEI/ISIN data

RMI handes us data already matched with LEIs and ISINs.  Other lists of company names may require us to stitch that together manually.

In [6]:
# TODO: sort why some notorious utilities are missing LEIs in the following query--bad source data?
rmi_lei_isin = pd.read_sql(f"select DISTINCT parent_name, parent_lei, parent_isin from {rmi_schema}.utility_information where parent_name IS NOT NULL", engine)
# Fabricate LEIs for entities that have none
print(sorted(list(rmi_lei_isin.loc[rmi_lei_isin.parent_lei.isnull()].parent_name)))
rmi_lei_isin.loc[rmi_lei_isin.parent_lei.isnull(), 'parent_lei'] = rmi_lei_isin.apply(lambda x: f"RMI{x.name:017}", axis=1)
rmi_lei_isin.loc[rmi_lei_isin.parent_isin.isnull(), 'parent_isin'] = rmi_lei_isin.apply(lambda x: f"ZZ{x.name:011}", axis=1)
# Install LEIs whose hierarchy levels don't match what we matched for SEC DERA data
rmi_lei_isin.loc[rmi_lei_isin.parent_name=='PG&E Corp.', 'parent_lei'] = '8YQ2GSDWYZXO2EDN3511'
rmi_lei_isin.loc[rmi_lei_isin.parent_name=='Verso Corp.', 'parent_lei'] = '549300FODXCTQ8DGT594'
rmi_lei_isin.loc[rmi_lei_isin.parent_name=='Verso Corp.', 'parent_isin'] = 'US92531L2079'
rmi_lei_dict = dict(zip(rmi_lei_isin.parent_lei, rmi_lei_isin.parent_isin))

['AJO Improvement Co.', 'Access Energy Coop.', 'Alfalfa Electric Coop., Inc.', 'Beacon Power Corp.', 'Beebee Island Corp.', 'Berea College Utilities', 'Bridger Valley Electric Association, Inc.', 'California Power Exchange Corp.', 'Century Power Corp.', 'Chugach Electric Assn Inc.', 'Citizens Electric Corp.', "Citizens' Electric Co.", 'Connexus Energy', 'Cross Sound Cable Co., LLC', 'Deseret Generation & Tran Coop.', 'Dixie-Escalante Rural Electric Association, Inc.', 'Edison Sault Electric Co.', 'Fall River Electric Coop.', 'First Electric Coop. Corp.', 'Florida Keys Electric Coop. Assoc., Inc.', 'Flowell Electric Association, Inc.', 'Graham County Electric Coop., Inc.', 'Great Bay Power Corp.', 'Gridliance Gp, LLC', 'Hagerstown Light Department', 'Harmony Florida Solar, LLC', 'Harney Electric Coop.', 'Hermiston Generating Co., L.P.', 'Idaho County Light & Power Coop. Assoc. Inc.', 'Indeck Energy Services, Inc.', 'Inland Power & Light Co.', 'Kandiyohi  Power Coop.', 'Lyon Rural Electr

Implement an *ad hoc* ingestion pipeline for Steel portfolio.  Later we will ingest steel production data.  We use this only to define the universe, not for actual investment information.

In [7]:
steel_idx = pd.read_csv(os.environ.get('PWD')+f"/itr-data-pipeline/data/external/mdt-steel-portfolio.csv",
                        header=0, sep=';', usecols=['company_name','company_lei','company_id'], dtype=str, engine='c')
#display(steel_idx)

Prepare GLEIF matching data for SEC DERA data.  In the future, such matching will use the ESG Entity-Matching pipeline (https://github.com/os-climate/financial-entity-cleaner/tree/version_0.1.0).

In [8]:
gleif_file = s3_source.Object(os.environ['S3_LANDING_BUCKET'],'mtiemann-GLEIF/DERA-matches.csv')
gleif_file.download_file(f'/tmp/dera-gleif.csv')
gleif_df = pd.read_csv(f'/tmp/dera-gleif.csv', header=0, sep=',', dtype=str, engine='c')
gleif_dict = dict(zip(gleif_df.name, gleif_df.LEI))
del(gleif_df)

# Many of the following ISINs are bonds, but some are also stocks (on various exchanges)
# But we don't need to load and match here, because the portfolio has the ISINs
if False:
    gleif_isin_file = s3_source.Object(os.environ['S3_LANDING_BUCKET'],'mtiemann-GLEIF/ISIN_LEI_20211009.csv')
    gleif_isin_file.download_file(f'/tmp/ISIN_LEI_20211009.csv')
    gleif_isins = pd.read_csv(f'/tmp/ISIN_LEI_20211009.csv', header=0, sep=',', dtype=str, engine='c')

Create a very simple entity matcher, cleaning up slight variations in company names between RMI's entity names, the SEC's entity names, and GLEIF's entity names.

Commented out are names we would have to fix if there were SEC data for them.  But because not, we'll never match what's not there in the first place.

In [9]:
for name, lei in zip(rmi_lei_isin.loc[rmi_lei_isin.parent_lei.str.startswith('RMI')]['parent_name'],
                     rmi_lei_isin.loc[rmi_lei_isin.parent_lei.str.startswith('RMI')]['parent_lei']):
    gleif_dict[name.upper()] = lei

In [10]:
# gleif_dict['Basin Electric Power Coop'.upper()] = gleif_dict['BASIN ELECTRIC POWER COOPERATIVE']
# gleif_dict['Big Rivers Electric Corp'.upper()] = gleif_dict['BIG RIVERS ELECTRIC CORPORATION']
gleif_dict['CHUGACH ELECTRIC ASSOCIATION INC'] = gleif_dict['CHUGACH ELECTRIC ASSN INC.']
gleif_dict['Cleco Partners LP'.upper()] = gleif_dict['CLECO CORPORATE HOLDINGS LLC']
# gleif_dict['Golden Spread Electric Coop., Inc'.upper()] = gleif_dict['GOLDEN SPREAD ELECTRIC COOPERATIVE, INC.']
gleif_dict['MIDWEST ENERGY INC'] = '549300O4B5CVWMKUES27'
gleif_dict['OG&E Energy'.upper()] = gleif_dict['OGE ENERGY CORP.']
# gleif_dict['Ohio Valley Electric Corp'.upper()] = gleif_dict['OHIO VALLEY ELECTRIC CORPORATION']
gleif_dict['Old Dominion Electric Coop'.upper()] = gleif_dict['OLD DOMINION ELECTRIC COOPERATIVE']
gleif_dict['PG&E Corp.'.upper()] = gleif_dict['PG&E CORP']
gleif_dict['Tri-State Generation & Transmission Association'.upper()] = gleif_dict['TRI-STATE GENERATION & TRANSMISSION ASSOCIATION, INC.']
gleif_dict['DOMINION ENERGY INC'] = 'ILUL7B6Z54MRYCF6H308'

gleif_dict['GROUP SIMEC SA DE CV'] = '529900LCYCXPA0TZEU09'

gleif_1 = { k.split(',')[0].split(' ')[0]:v for k,v in gleif_dict.items() }
gleif_2 = { ' '.join(k.split(',')[0].split(' ')[0:2]):v for k,v in gleif_dict.items() }

def gleif_match(x):
    x = x.split(',')[0]
    if x in gleif_dict:
        return gleif_dict[x]
    x = x.replace('.','')
    if x in gleif_dict:
        return gleif_dict[x]
    x2 = ' '.join(x.split(' ')[0:2])
    if x2 in gleif_2:
        return gleif_2[x2]
    if ' ' not in x and x in gleif_1:
        return gleif_1[x]
    return None

Collect the universe of company names for the sectors we cover.  Steel sector is SIC 3310-3317. Electricity Utilities is SIC 4911 (but also 4931-4932 and 4991).

Some conglomerates have more general SIC codes that hide their activities in sectors of interest.  Others report those SIC codes within reportable segements.
Without more detailed SEC DERA data (available in an S3 bucket but not yet processed as a pipeline), we will not collect the company names we need to collect.

In [11]:
sec_lei_isin = pd.read_sql(f"""
select DISTINCT F.name, F.lei, F.sic
from {dera_schema}.financials_by_lei F
where (sic=4911 or sic=4931 or sic=4932 or sic=4991)
      or (sic>=3310 and sic<=3317)
""", engine)
sec_lei_isin.lei = sec_lei_isin.name.map(gleif_dict).fillna(sec_lei_isin.lei)
sec_lei_map = {
    'DOMINION ENERGY INC':'ILUL7B6Z54MRYCF6H308',
    'GROUP SIMEC SA DE CV':'529900LCYCXPA0TZEU09',
    'CLEANSPARK, INC.':'254900VO7KBRJQDGY810',
    'ENEL GENERACION CHILE S.A.':'549300PVHXUFEIE6LY50',
    'ARCHAEA ENERGY INC.':'549300KT0ZQUIBAX8U21',
    'ENERGY CO OF MINAS GERAIS':'254900W703PXLDSEM056',
    'PAMPA ENERGY INC.':'254900QNIK0CVURGML24',
    'CPFL ENERGY INC':'529900GBWSBDXN8GGM28'
}
sec_lei_isin.lei = sec_lei_isin.name.map(sec_lei_map).fillna(sec_lei_isin.lei)

missing_leis = sec_lei_isin[sec_lei_isin.lei.isna()]

In [12]:
missing_leis.name

22                   PREMIER HOLDING CORP.
37                             UNITIL CORP
45     HUADI INTERNATIONAL GROUP CO., LTD.
61                 FRIEDMAN INDUSTRIES INC
78        ENERGY CONVERSION SERVICES, INC.
94                MONTAUK RENEWABLES, INC.
104                          VETANOVA INC.
105                 PECK CO HOLDINGS, INC.
110              OSSEN INNOVATION CO. LTD.
125              OCEAN THERMAL ENERGY CORP
130            BRAZILIAN ELECTRIC POWER CO
146                         HELIOGEN, INC.
159             8POINT3 ENERGY PARTNERS LP
Name: name, dtype: object

In [13]:
sec_lei_isin.dropna(inplace=True)
print("The following companies are missing LEI information and will be dropped:")
display(missing_leis)

The following companies are missing LEI information and will be dropped:


,name,lei,sic
22,PREMIER HOLDING CORP.,None,4911
37,UNITIL CORP,None,4931
45,"HUADI INTERNATIONAL GROUP CO., LTD.",None,3317
61,FRIEDMAN INDUSTRIES INC,None,3310
78,"ENERGY CONVERSION SERVICES, INC.",None,4911
94,"MONTAUK RENEWABLES, INC.",None,4932
104,VETANOVA INC.,None,4911
105,"PECK CO HOLDINGS, INC.",None,4932
110,OSSEN INNOVATION CO. LTD.,None,3312
125,OCEAN THERMAL ENERGY CORP,None,4931


We create a theoretical portfolio that conveniently contains all available LEI and ISIN information, meaning we don't need to do entity matching or ISIN matching.

Other portfolios may need a lot more work before they can be used to precompute other data.  The code above are samples of the kind of extra data/processing needed for such portfolios.

In [14]:
rmi_idx = rmi_lei_isin.rename(columns={'parent_name':'company_name', 'parent_lei':'company_lei', 'parent_isin':'company_id'})
# rmi_idx.insert(1, 'company_lei', portfolio_df.company_name.str.upper().map(gleif_match))
# if rmi_idx.company_lei.isna().any():
#     display(rmi_idx[rmi_idx.company_lei.isna()])
rmi_idx.loc[rmi_idx.company_id.isna(), 'company_id'] = rmi_idx.apply(lambda x: f"ZZ{x.name:010}", axis=1)

print(f"Number of RMI portfolio copmanies = {len(rmi_idx)}")

Number of RMI portfolio copmanies = 183


Show list of RMI companies that use made-up LEIs or ISINs

In [15]:
rmi_idx[rmi_idx.company_lei.str.startswith('RMI')]

,company_name,company_lei,company_id
2,"New York Transco, LLC",RMI00000000000000002,ZZ00000000002
4,Harney Electric Coop.,RMI00000000000000004,ZZ00000000004
5,Idaho County Light & Power Coop. Assoc. Inc.,RMI00000000000000005,ZZ00000000005
6,Lyon Rural Electric Coop.,RMI00000000000000006,ZZ00000000006
7,"Salmon River Electric Coop., Inc.",RMI00000000000000007,ZZ00000000007
...,...,...,...
170,"North Central Missouri Electric Coop., Inc.",RMI00000000000000170,ZZ00000000170
173,"Graham County Electric Coop., Inc.",RMI00000000000000173,ZZ00000000173
176,"Bridger Valley Electric Association, Inc.",RMI00000000000000176,ZZ00000000176
177,Mcdonough Power Coop.,RMI00000000000000177,ZZ00000000177


Add Steel company portfolio

In [16]:
portfolio_idx = pd.concat([rmi_idx, steel_idx])
portfolio_idx = portfolio_idx.convert_dtypes()

print(f"Number of total portfolio companies = {len(portfolio_idx)}")

Number of total portfolio companies = 208


### Company Data

The SIC-to-ISIC table is an open workstream item: https://github.com/os-climate/itr-data-pipeline/issues/1

### Capture a list of the companies for which we have good financial info

We limit our view to the companies in our portfolio.  The user can prioritize whether this is the best source of revenue, market cap, etc., or whether they prefer another source.

Note for future reference: Berkshire Hathaway has one line of business for Energy and another for Steel.  We don't yet have line-of-business info because we use summary data from SEC DERA, not the detailed Notes version of the dataset.

In [17]:
portfolio_idx

,company_name,company_lei,company_id
0,FirstEnergy Corp.,549300SVYJS666PQJH88,US3379321074
1,Sempra,PBBKGKLRK5S5C0Y4T545,US8168511090
2,"New York Transco, LLC",RMI00000000000000002,ZZ00000000002
3,Black Hills Corp.,3MGELCRSTNSAMJ962671,US0921131092
4,Harney Electric Coop.,RMI00000000000000004,ZZ00000000004
...,...,...,...
20,TIMKENSTEEL CORP,549300QZTZWHDE9HJL14,US8873991033
21,TITAN INTERNATIONAL INC,254900CXRGBE7C4B5A06,US88830M1027
22,UNITED STATES STEEL CORP,JNLUVFYJT1OZSIQ24U47,US9129091081
23,UNIVERSAL STAINLESS & ALLOY PRODUCTS INC,5493001OEIZDUGXZDE09,US9138371003


In [18]:
ingest_table = 'portfolio_universe'

drop_table = engine.execute(f"drop table if exists {ingest_schema}.{ingest_table}")
drop_table.fetchall()

columnschema = osc.create_table_schema_pairs(portfolio_idx, typemap={"datetime64[ns]":"timestamp(6)"})

tabledef = f"""
create table if not exists {ingest_catalog}.{ingest_schema}.{ingest_table}(
{columnschema}
) with (
    format = 'ORC',
    partitioning = array['bucket(company_lei, 20)']
)
"""
print(tabledef)
create_table = engine.execute(tabledef)
print(create_table.fetchall())
portfolio_idx.to_sql(ingest_table,
                     con=engine, schema=ingest_schema, if_exists='append',
                     index=False,
                     method=osc.TrinoBatchInsert(batch_size = 5000, verbose = True))


create table if not exists osc_datacommons_dev.sandbox.portfolio_universe(
    company_name varchar,
    company_lei varchar,
    company_id varchar
) with (
    format = 'ORC',
    partitioning = array['bucket(company_lei, 20)']
)

[(True,)]
constructed fully qualified table name as: "sandbox.portfolio_universe"
inserting 208 records
  ('FirstEnergy Corp.', '549300SVYJS666PQJH88', 'US3379321074')
  ('Sempra', 'PBBKGKLRK5S5C0Y4T545', 'US8168511090')
  ('New York Transco, LLC', 'RMI00000000000000002', 'ZZ00000000002')
  ...
  ('WORTHINGTON INDUSTRIES INC', '1WRCIANKYOIK6KYE5E82', 'US9818111026')
batch insert result: [(208,)]


### Create a list with metric labels embedded in the output for easy reading...

Highlight any rows that have NULL data

### Capture and print a list of companies with financial info

Financial information is part of the "fundamental data" we need for the ITR portfolio companies.  The other part is base year production, emission, and intensity data.  We query the two separately because we have a unified source of truth for the former (SEC DERA) but multiple sources for the latter (RMI for Electric Utilities and MDT for Steel).

### Financial info:
* Company Name, LEI, ISIN, year
* ISIC Code (for Sector)
* Country and Region
* Revenue, Market Cap, Enterprise Value, Assets, Cash

We currently focus exclusively on data from 2019 as our base year

In [19]:
base_financial_sql = f"""
select DISTINCT P.company_name, P.company_lei, P.company_id,
       F.country, UN.region_ar6_10 as region,
       if(S2I.isic=2410 or P.company_name='CLEVELAND-CLIFFS INC', 'Steel', 'Electricity Utilities') as sector,
       'equity' as exposure, 'USD' as currency,
       year(F.ddate) as year,
       F.market_cap_usd as company_market_cap,
       F.revenue_usd as company_revenue,
       F.market_cap_usd+F.debt_usd-F.cash_usd as company_ev,
       F.market_cap_usd+F.debt_usd as company_evic,
       F.assets_usd as company_total_assets,
       F.cash_usd as company_cash_equivalents,
       F.debt_usd as company_debt
from {ingest_schema}.portfolio_universe as P
     left join {dera_schema}.financials_by_lei as F on F.lei=P.company_lei and year(F.ddate)=2019
     join {iso3166_schema}.countries as I on F.country=I.alpha_2
     join {essd_schema}.regions as UN on I.alpha_3=UN.iso
     -- join {dera_schema}.{dera_prefix}sub as S on S.cik=F.cik
     -- left join {rmi_schema}.utility_information as U on U.parent_lei=P.company_lei
     -- left join {gleif_schema}.gleif_isin_lei G on G.lei=P.lei and G.isin=U.parent_isin
     left join {dera_schema}.sic_isic as S2I on S2I.sic=F.sic
     -- left join {rmi_schema}.operations_emissions_by_fuel as E on U.respondent_id=E.respondent_id and year(E.year)=year(F.ddate)
-- where E.owned_or_total='owned'
group by P.company_name, P.company_lei, P.company_id,
       F.country, UN.region_ar6_10,
       if(S2I.isic=2410 or P.company_name='CLEVELAND-CLIFFS INC', 'Steel', 'Electricity Utilities'),
       6, 7, -- exposure, currency
       year(F.ddate),
       F.market_cap_usd, F.revenue_usd, F.market_cap_usd+F.debt_usd-F.cash_usd, F.market_cap_usd+F.debt_usd, F.assets_usd, F.cash_usd, F.debt_usd
order by P.company_name
limit 200
"""

### Emissions/Production info
* Company Name, LEI, ISIN (join axis with financial info)
* Sector (inferred from RMI data as a source rather than ISIC)
* Production (in whatever units -- we need units in either metadata or a column or as part of the data element iselft)
* S1, S2, S3 emissions (in megametric tons CO2e)
* S1, S2, S3 emissions intensity (emissions / production, in whatever units this resolves to)

We currently focus exclusively on data from 2019 as our base year

Note that RMI data is S1 only (own generation); we use zero as S2 value

In [20]:
# 'sector', 's1_co2', 's2_co2', 's3_co2', 's1_ei', 's2_ei', 's3_ei', 'production'
scopes = ['s1', 's2', 's3']

emissions_sql = f"""
select DISTINCT P.company_name, P.company_lei, P.company_id,
       'Electricity Utilities' as sector, year(E.year) as year,
       sum(E.emissions_co2 + (265/1000000.0)*coalesce(E.emissions_nox, 0)) as ghg_s1, 0 as ghg_s2, NULL as ghg_s3,
       sum(E.emissions_co2 + (265/1000000.0)*coalesce(E.emissions_nox, 0)) / sum(E.generation) as ei_s1, 0 as ei_s2, NULL as ei_s3,
       sum(E.generation) as production
from {ingest_schema}.portfolio_universe as P
     join {rmi_schema}.utility_information as U on U.parent_lei=P.company_lei
     join {rmi_schema}.operations_emissions_by_fuel as E on U.respondent_id=E.respondent_id
where year(E.year)>=2014 and year(E.year)<2023
   and P.company_lei!='529900L26LIS2V8PWM23' -- American States Water has negative/zero production values that mess things up
-- and E.owned_or_total='owned'
group by P.company_name, P.company_lei, P.company_id, 3, year(E.year)
order by P.company_name
"""

### `financial_df` contains all the base year (2019) financial, production, and emissions data

For now our benchmark data covers only North America and Europe.  Over time, we expect additional regions (possibly on a per-sector basis).

In [21]:
financial_df = pd.read_sql(base_financial_sql, engine, index_col=['company_name', 'company_lei', 'company_id', 'sector']).convert_dtypes()
financial_df.region = financial_df.region.apply(lambda x: x if x in ['Asia', 'Europe', 'North America'] else 'Global').astype('string')
financial_df

,,,,country,region,exposure,currency,year,company_market_cap,company_revenue,company_ev,company_evic,company_total_assets,company_cash_equivalents,company_debt
company_name,company_lei,company_id,sector,,,,,,,,,,,,
AES Corp.,2NUNNB7D43COUIRE5295,US00130H1059,Electricity Utilities,US,North America,equity,USD,2019,10870000000.0,10189000000.0,10102000000,11131000000.0,33648000000.0,1029000000.0,261000000.0
"ALLETE, Inc.",549300NNLSIMY6Z8OT86,US0185223007,Electricity Utilities,US,North America,equity,USD,2019,4285299935.0,1240500000.0,5829799935,5899099935.0,5482800000.0,69300000.0,1613800000.0
Alcoa Corp.,549300T12EZ1F6PWWU29,US0138721065,Electricity Utilities,US,North America,equity,USD,2019,4300000000.0,10433000000.0,5221000000,6100000000.0,14631000000.0,879000000.0,1800000000.0
Algonquin Power & Utilities Corp.,549300K5VIUTJXQL7X75,US0158577090,Electricity Utilities,CA,North America,equity,USD,2019,<NA>,1624921000.0,<NA>,<NA>,10911470000.0,62485000.0,6500799000.0
Alliant Energy,5493009ML300G373MZ12,US0188021085,Electricity Utilities,US,North America,equity,USD,2019,11600000000.0,3647700000.0,18503600000,18519900000.0,16700700000.0,16300000.0,6919900000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Verso Corp.,549300FODXCTQ8DGT594,US92531L2079,Electricity Utilities,US,North America,equity,USD,2019,658075983.0,2444000000.0,622075983,664075983.0,1721000000.0,42000000.0,6000000.0
Vistra Corp.,549300KP43CPCUJOOG15,US92840M1027,Electricity Utilities,US,North America,equity,USD,2019,8654325784.0,11809000000.0,18456325784,18756325784.0,26616000000.0,300000000.0,10102000000.0
WEC Energy Group,549300IGLYTZUK3PVP70,US92939U1060,Electricity Utilities,US,North America,equity,USD,2019,26300000000.0,7523100000.0,38120800000,38158300000.0,34951800000.0,37500000.0,11858300000.0


### `emissions_df` contains all the base year (2019) production and emissions data

In [22]:
rmi_emissions_df = pd.read_sql(emissions_sql, engine, index_col=['company_name', 'company_lei', 'company_id', 'sector']).convert_dtypes()
for scope in scopes:
    rmi_emissions_df["ghg_" + scope] = rmi_emissions_df["ghg_" + scope].astype('pint[Mt CO2]')
    rmi_emissions_df["ei_" + scope] = rmi_emissions_df["ei_" + scope].astype('pint[Mt CO2/TWh]')
rmi_emissions_df["production"] = rmi_emissions_df["production"].astype('pint[TWh]')
rmi_emissions_df["ghg_s1s2"] = rmi_emissions_df["ghg_s1"] + rmi_emissions_df["ghg_s2"]
rmi_emissions_df["ei_s1s2"] = rmi_emissions_df["ghg_s1s2"] / rmi_emissions_df["production"]
template_rmi_df = rmi_emissions_df.pivot(index=None, columns='year')

# Put column names into YYYY_metric order (Multi-index has this order inverted)
template_rmi_df.columns = template_rmi_df.columns.map(lambda x: f"{x[1]}_{x[0]}")
template_rmi_df = template_rmi_df.loc[:, ~template_rmi_df.columns.str.contains('_ei_')]
display(template_rmi_df)

/Users/michael/opt/miniconda3/envs/jupyter/lib/python3.10/site-packages/pint_pandas/pint_array.py:194: RuntimeWarning: pint-pandas does not support magnitudes of <class 'numpy.int64'>. Converting magnitudes to float.
  warnings.warn(
/Users/michael/opt/miniconda3/envs/jupyter/lib/python3.10/site-packages/pint_pandas/pint_array.py:194: RuntimeWarning: pint-pandas does not support magnitudes of <class 'NoneType'>. Converting magnitudes to float.
  warnings.warn(
/Users/michael/opt/miniconda3/envs/jupyter/lib/python3.10/site-packages/pint_pandas/pint_array.py:648: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  return np.array(qtys, dtype="object", copy=copy)
/Users/michael/opt/miniconda3/envs/jupyter/lib/python3.10/site-packages/pint_pandas/pint_array.py:648: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  return np.array(qtys, dtype="object", copy=copy)


,,,,2014_ghg_s1,2015_ghg_s1,2016_ghg_s1,2017_ghg_s1,2018_ghg_s1,2019_ghg_s1,2020_ghg_s1,2014_ghg_s2,2015_ghg_s2,2016_ghg_s2,...,2018_production,2019_production,2020_production,2014_ghg_s1s2,2015_ghg_s1s2,2016_ghg_s1s2,2017_ghg_s1s2,2018_ghg_s1s2,2019_ghg_s1s2,2020_ghg_s1s2
company_name,company_lei,company_id,sector,,,,,,,,,,,,,,,,,,,,,
AES Corp.,2NUNNB7D43COUIRE5295,US00130H1059,Electricity Utilities,27.647741534207274,22.07553373053335,20.952695203741783,10.483392150493385,11.235889892304938,11.616368146942126,9.425520125987727,0.0,0.0,0.0,...,13.537873157079574,15.292476505000003,13.075167509999998,27.647741534207274,22.07553373053335,20.952695203741783,10.483392150493385,11.235889892304938,11.616368146942126,9.425520125987727
"ALLETE, Inc.",549300NNLSIMY6Z8OT86,US0185223007,Electricity Utilities,9.31812469169611,8.454256540384762,8.028791524631679,6.566070166735715,6.622018867468628,4.223366148933545,3.750731560742501,0.0,0.0,0.0,...,8.743458000000002,6.490906199999999,6.078342399999998,9.31812469169611,8.454256540384762,8.028791524631679,6.566070166735715,6.622018867468628,4.223366148933545,3.750731560742501
Alcoa Corp.,549300T12EZ1F6PWWU29,US0138721065,Electricity Utilities,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0692539999999997,1.0264219999999995,1.1960759999999995,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Algonquin Power & Utilities Corp.,549300K5VIUTJXQL7X75,US0158577090,Electricity Utilities,2.973143721429275,2.980086247514266,3.4276488419499493,3.972490599757491,3.768993147181291,3.3272864848948975,2.4089141024115746,0.0,0.0,0.0,...,6.3116767568,5.314575956799999,4.588301243199998,2.973143721429275,2.980086247514266,3.4276488419499493,3.972490599757491,3.768993147181291,3.3272864848948975,2.4089141024115746
Alliant Energy,5493009ML300G373MZ12,US0188021085,Electricity Utilities,13.85605168211895,14.674657780360885,12.247083817892092,13.595806479490603,14.580423815329834,11.098764958614002,11.037755852936693,0.0,0.0,0.0,...,21.667851863000006,20.524337211500004,22.008184385999993,13.85605168211895,14.674657780360885,12.247083817892092,13.595806479490603,14.580423815329834,11.098764958614002,11.037755852936693
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WEC Energy Group,549300IGLYTZUK3PVP70,US92939U1060,Electricity Utilities,21.96727443074169,22.45967009728603,19.020371447730597,19.6272318151656,16.478434173257213,9.874576584949224,9.667297788165046,0.0,0.0,0.0,...,21.139317437999996,15.874233993999985,15.865285688499995,21.96727443074169,22.45967009728603,19.020371447730597,19.6272318151656,16.478434173257213,9.874576584949224,9.667297788165046
"Wabash Valley Power Assn, Inc",VR27ZYPWHGW7Z1BM8Y69,ZZ00000000046,Electricity Utilities,1.7436830993754122,1.3246810675271066,1.2045424632467225,1.0210695904332496,1.2545689587566193,0.9139534626802568,1.013641719006578,0.0,0.0,0.0,...,1.6959904569333326,1.3141707926666668,1.4430816797333326,1.7436830993754122,1.3246810675271066,1.2045424632467225,1.0210695904332496,1.2545689587566193,0.9139534626802568,1.013641719006578
Wolverine Power Supply Coop.,549300ROWOIV5X5MB591,ZZ00000000097,Electricity Utilities,0.13588491303774736,0.273640514008736,0.5427537046781931,0.5331105539006246,0.8078006096815795,0.542313592299735,0.8251582863266445,0.0,0.0,0.0,...,1.210923649308789,0.7823584383376859,1.2915553571686702,0.13588491303774736,0.273640514008736,0.5427537046781931,0.5331105539006246,0.8078006096815795,0.542313592299735,0.8251582863266445


### Collect emissions/production info from the MDT Steel data
* Company Name, LEI, ISIN (join axis with financial info)
* Sector (inferred as Steel from source)
* Production (in whatever units -- we need units in either metadata or a column or as part of the data element itself)
* S1, S2, S3 emissions (in whatever units of CO2e)
* S1, S2, S3 emissions intensity (emissions / production, in whatever units this resolves to)

If a company has no emissions or production information, we don't carry it forward as data (even if it does have revenue, earnings, etc.)

In [23]:
steel_wb = pd.read_excel(os.environ.get('PWD')+f"/itr-data-pipeline/data/external/mdt-steel-demo.xlsx", sheet_name=None)
steel_production = steel_wb['Steel Fe_tons'].dropna(axis=1,how='all')
steel_production.set_index(steel_production.columns[0:3].to_list(), inplace=True)
steel_production = steel_production.dropna(axis=0,how='all')
steel_production = steel_production.astype("pint[Fe_ton]")
steel_co2 = {}
steel_ei = {}
for scope in scopes:
    steel_co2[scope] = steel_wb[f"Steel CO2e {scope.upper()}"].dropna(axis=1,how='all')
    steel_co2[scope].set_index(steel_co2[scope].columns[0:3].to_list(), inplace=True)
    steel_co2[scope] = steel_co2[scope].dropna(axis=0,how='all')
    steel_co2[scope] = steel_co2[scope].astype("pint[t CO2]")
    steel_ei[scope] = (steel_co2[scope] / steel_production).dropna(how='all')

In [24]:
def rename_column_emissions(df, scope):
    df = df.loc[:, 2014:2020]
    df.columns = df.columns.map(lambda x: f"{x}_ghg_{scope}")
    return df

template_steel_co2 = pd.concat([rename_column_emissions(steel_co2[scope], scope) for scope in scopes], axis=1)
for year in range(2014,2021):
    template_steel_co2.insert(len(template_steel_co2.columns)-5,f"{year}_ghg_s1s2", steel_co2['s1'][year]+steel_co2['s2'][year])
template_steel_co2

/Users/michael/opt/miniconda3/envs/jupyter/lib/python3.10/site-packages/pint_pandas/pint_array.py:648: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  return np.array(qtys, dtype="object", copy=copy)
/Users/michael/opt/miniconda3/envs/jupyter/lib/python3.10/site-packages/pint_pandas/pint_array.py:648: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  return np.array(qtys, dtype="object", copy=copy)


,,,2014_ghg_s1,2015_ghg_s1,2016_ghg_s1,2017_ghg_s1,2018_ghg_s1,2019_ghg_s1,2020_ghg_s1,2014_ghg_s2,2015_ghg_s2,2016_ghg_s2,...,2016_ghg_s1s2,2017_ghg_s1s2,2018_ghg_s1s2,2019_ghg_s1s2,2020_ghg_s1s2,2016_ghg_s3,2017_ghg_s3,2018_ghg_s3,2019_ghg_s3,2020_ghg_s3
company_name,company_lei,company_id,,,,,,,,,,,,,,,,,,,,,
CARPENTER TECHNOLOGY CORP,DX6I6ZD3X5WNNCDJKP85,US1442851036,298055.0,298055.0,298055.0,298055.0,298055.0,298055.0,292832.15098729834,660000.0,660000.0,660000.0,...,958055.0,958055.0,958055.0,958055.0,951267.1509872983,nan,nan,nan,nan,nan
CLEVELAND-CLIFFS INC,549300TM2WLI2BJMDD86,US1858991011,35098923.07076,32771887.7758,33209464.625,32357763.7366,31034981.66376,30349904.4497999,25607731.879518665,4494608.671214038,4517098.2923521735,4431504.715648127,...,37640969.34064813,36830868.09318798,35448337.033455685,34776009.68576185,29279098.879518665,1934075.55568,2449774.834,2449865.646,2194701.704,1851779.56275
COMMERCIAL METALS CO,549300OQS2LO07ZJ7N73,US2017231034,1048006.0,1048006.0,1048006.0,1048006.0,1048006.0,1048006.0,1106156.0,2548437.0,2548437.0,2548437.0,...,3596443.0,3596443.0,3596443.0,2548437.0,2572986.0,nan,nan,nan,nan,nan
GERDAU S.A.,254900YDV6SEQQPZVG24,US3737371050,12075000.0,12075000.0,12075000.0,12075000.0,10707412.125,9056519.0,9198407.0,4025000.0,4025000.0,4025000.0,...,16100000.0,16100000.0,14276549.5,11947505.0,11280922.0,nan,nan,nan,nan,nan
NIPPON STEEL CORP,35380065QWQ4U2V3PA33,JP3381000003,80501000.0,80501000.0,80501000.0,80501000.0,81099000.0,78384000.0,62860000.0,12478000.0,12478000.0,12478000.0,...,92979000.0,92979000.0,93662000.0,90262000.0,73706000.0,20957000.0,20957000.0,21191000.0,20937000.0,18078000.0
NUCOR CORP,549300GGJCRSI2TIEJ46,US6703461052,4800000.0,4800000.0,4800000.0,4800000.0,4800000.0,4400000.0,4700000.0,5785714.285714285,5785714.285714285,5785714.285714285,...,10585714.285714285,10585714.285714285,10585714.285714285,9800000.0,10100000.0,7557446.808510638,7557446.808510638,7557446.808510638,6927659.574468086,7400000.0
POSCO,988400E5HRVX81AYLM04,KR7005490008,84412800.0,82741300.0,81309800.0,75633360.0,77391479.0,79447924.0,75069656.0,4741000.0,4430700.0,3715700.0,...,85025500.0,76741041.0,78498443.0,80263890.0,75649882.0,18044000.0,18044000.0,18903000.0,13139000.0,11951000.0
STEEL DYNAMICS INC,549300HGGKEL4FYTTQ83,US8581191009,3215942.0,3215942.0,3215942.0,3215942.0,3299883.0,3145097.0,3063829.9454545453,1700245.0,1700245.0,1700245.0,...,4916187.0,4916187.0,5162928.0,4889766.0,4758090.081818182,nan,nan,nan,nan,nan
TENARIS SA,549300Y7C05BKC4HZB40,US88031M1099,2000000.0,2000000.0,2000000.0,2000000.0,2000000.0,1800000.0,1100000.0,1000000.0,1000000.0,1000000.0,...,3000000.0,3000000.0,3000000.0,2600000.0,1500000.0,3200000.0,3200000.0,3200000.0,2300000.0,1300000.0


In [25]:
template_steel_production = steel_production.loc[:, 2014:2020]
template_steel_production.columns = template_steel_production.columns.map(lambda x: f"{x}_production")
template_steel_production

/Users/michael/opt/miniconda3/envs/jupyter/lib/python3.10/site-packages/pint_pandas/pint_array.py:648: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  return np.array(qtys, dtype="object", copy=copy)
/Users/michael/opt/miniconda3/envs/jupyter/lib/python3.10/site-packages/pint_pandas/pint_array.py:648: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  return np.array(qtys, dtype="object", copy=copy)


,,,2014_production,2015_production,2016_production,2017_production,2018_production,2019_production,2020_production
company_name,company_lei,company_id,,,,,,,
AK STEEL HOLDING CORP,529900DT4E7ZNETMVC04,US0015471081,6132700.0,7089200.0,6051800.0,5596200.0,5683400.0,5342200.0,5422332.999999999
ARCELORMITTAL,2EULGUTUI56JI9SAL165,LU0140205948,85100000.0,84600000.0,83900000.0,85200000.0,83900000.0,84500000.0,69100000.0
CARPENTER TECHNOLOGY CORP,DX6I6ZD3X5WNNCDJKP85,US1442851036,138831.0,138831.0,138831.0,138831.0,138831.0,138831.0,140944.858159875
CLEVELAND-CLIFFS INC,549300TM2WLI2BJMDD86,US1858991011,91232700.0,91689200.0,89951800.0,90796200.0,89583400.0,89842200.0,74522333.0
COMMERCIAL METALS CO,549300OQS2LO07ZJ7N73,US2017231034,5301216.0,5301216.0,5301216.0,5301216.0,5301216.0,5301216.0,5543677.0
GERDAU S.A.,254900YDV6SEQQPZVG24,US3737371050,16100000.0,16100000.0,16100000.0,16100000.0,14276549.5,12453099.0,13142354.3
NIPPON STEEL CORP,35380065QWQ4U2V3PA33,JP3381000003,49580000.0,49580000.0,49580000.0,49580000.0,48500000.0,45890000.0,36630000.0
NUCOR CORP,549300GGJCRSI2TIEJ46,US6703461052,22500000.0,22500000.0,22500000.0,22500000.0,22500000.0,20700000.0,20300000.0
POSCO,988400E5HRVX81AYLM04,KR7005490008,41428000.0,42027000.0,42199000.0,37207000.0,37735000.0,38007000.0,35935000.0


In [26]:
template_steel_df = pd.concat([template_steel_co2, template_steel_production], axis=1)
template_steel_df.insert(0, 'sector', 'Steel')
template_steel_df.set_index(['sector'], append=True, inplace=True)
template_steel_df.insert(0, 'emissions_metric', 't CO2')
template_steel_df.insert(1, 'production_metric', 'Fe_ton')
template_steel_df

/Users/michael/opt/miniconda3/envs/jupyter/lib/python3.10/site-packages/pint_pandas/pint_array.py:648: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  return np.array(qtys, dtype="object", copy=copy)
/Users/michael/opt/miniconda3/envs/jupyter/lib/python3.10/site-packages/pint_pandas/pint_array.py:648: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  return np.array(qtys, dtype="object", copy=copy)


,,,,emissions_metric,production_metric,2014_ghg_s1,2015_ghg_s1,2016_ghg_s1,2017_ghg_s1,2018_ghg_s1,2019_ghg_s1,2020_ghg_s1,2014_ghg_s2,...,2018_ghg_s3,2019_ghg_s3,2020_ghg_s3,2014_production,2015_production,2016_production,2017_production,2018_production,2019_production,2020_production
company_name,company_lei,company_id,sector,,,,,,,,,,,,,,,,,,,,,
CARPENTER TECHNOLOGY CORP,DX6I6ZD3X5WNNCDJKP85,US1442851036,Steel,t CO2,Fe_ton,298055.0,298055.0,298055.0,298055.0,298055.0,298055.0,292832.15098729834,660000.0,...,nan,nan,nan,138831.0,138831.0,138831.0,138831.0,138831.0,138831.0,140944.858159875
CLEVELAND-CLIFFS INC,549300TM2WLI2BJMDD86,US1858991011,Steel,t CO2,Fe_ton,35098923.07076,32771887.7758,33209464.625,32357763.7366,31034981.66376,30349904.4497999,25607731.879518665,4494608.671214038,...,2449865.646,2194701.704,1851779.56275,91232700.0,91689200.0,89951800.0,90796200.0,89583400.0,89842200.0,74522333.0
COMMERCIAL METALS CO,549300OQS2LO07ZJ7N73,US2017231034,Steel,t CO2,Fe_ton,1048006.0,1048006.0,1048006.0,1048006.0,1048006.0,1048006.0,1106156.0,2548437.0,...,nan,nan,nan,5301216.0,5301216.0,5301216.0,5301216.0,5301216.0,5301216.0,5543677.0
GERDAU S.A.,254900YDV6SEQQPZVG24,US3737371050,Steel,t CO2,Fe_ton,12075000.0,12075000.0,12075000.0,12075000.0,10707412.125,9056519.0,9198407.0,4025000.0,...,nan,nan,nan,16100000.0,16100000.0,16100000.0,16100000.0,14276549.5,12453099.0,13142354.3
NIPPON STEEL CORP,35380065QWQ4U2V3PA33,JP3381000003,Steel,t CO2,Fe_ton,80501000.0,80501000.0,80501000.0,80501000.0,81099000.0,78384000.0,62860000.0,12478000.0,...,21191000.0,20937000.0,18078000.0,49580000.0,49580000.0,49580000.0,49580000.0,48500000.0,45890000.0,36630000.0
NUCOR CORP,549300GGJCRSI2TIEJ46,US6703461052,Steel,t CO2,Fe_ton,4800000.0,4800000.0,4800000.0,4800000.0,4800000.0,4400000.0,4700000.0,5785714.285714285,...,7557446.808510638,6927659.574468086,7400000.0,22500000.0,22500000.0,22500000.0,22500000.0,22500000.0,20700000.0,20300000.0
POSCO,988400E5HRVX81AYLM04,KR7005490008,Steel,t CO2,Fe_ton,84412800.0,82741300.0,81309800.0,75633360.0,77391479.0,79447924.0,75069656.0,4741000.0,...,18903000.0,13139000.0,11951000.0,41428000.0,42027000.0,42199000.0,37207000.0,37735000.0,38007000.0,35935000.0
STEEL DYNAMICS INC,549300HGGKEL4FYTTQ83,US8581191009,Steel,t CO2,Fe_ton,3215942.0,3215942.0,3215942.0,3215942.0,3299883.0,3145097.0,3063829.9454545453,1700245.0,...,nan,nan,nan,8529969.0,8529969.0,8529969.0,8529969.0,9074135.0,8793160.0,8925057.399999999
TENARIS SA,549300Y7C05BKC4HZB40,US88031M1099,Steel,t CO2,Fe_ton,2000000.0,2000000.0,2000000.0,2000000.0,2000000.0,1800000.0,1100000.0,1000000.0,...,3200000.0,2300000.0,1300000.0,2900000.0,2900000.0,2900000.0,2900000.0,2900000.0,2900000.0,1800000.0


In [27]:
pd.options.display.max_rows = 99
pd.options.display.max_columns = 49
template_df = pd.concat([financial_df, pd.concat([template_steel_df, template_rmi_df])], axis=1).dropna(thresh=16).drop(columns=['company_cash_equivalents', 'company_debt'], axis=1)
template_df.loc[pd.IndexSlice[:, :, :, ['Electricity Utilities']], ['emissions_metric', 'production_metric']] = ['Mt CO2', 'TWh']
template_df = template_df.reset_index()
cols = template_df.columns.tolist()
cols = cols[:3] + cols[4:6] + [cols[3]] + cols[6:]
template_df = template_df[cols]
for col in cols:
    if col.startswith('2020_'):
        col_index = template_df.columns.get_loc(col)
        for year in [2022, 2021]:
            newcol = col.replace('2020', str(year))
            template_df.insert(col_index+1, newcol, np.nan)
            if '_ghg_' in newcol:
                template_df.loc[:, newcol] = template_df['emissions_metric'].map(lambda x: Q_(np.nan, x))
            else:
                template_df.loc[:, newcol] = template_df['production_metric'].map(lambda x: Q_(np.nan, x))
display(template_df.sample(15))
pd.reset_option("display.max_rows")
pd.reset_option("display.max_columns")

/Users/michael/opt/miniconda3/envs/jupyter/lib/python3.10/site-packages/pint_pandas/pint_array.py:648: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  return np.array(qtys, dtype="object", copy=copy)


,company_name,company_lei,company_id,country,region,sector,exposure,currency,year,company_market_cap,company_revenue,company_ev,company_evic,company_total_assets,emissions_metric,production_metric,2014_ghg_s1,2015_ghg_s1,2016_ghg_s1,2017_ghg_s1,2018_ghg_s1,2019_ghg_s1,2020_ghg_s1,2021_ghg_s1,...,2015_ghg_s1s2,2016_ghg_s1s2,2017_ghg_s1s2,2018_ghg_s1s2,2019_ghg_s1s2,2020_ghg_s1s2,2021_ghg_s1s2,2022_ghg_s1s2,2016_ghg_s3,2017_ghg_s3,2018_ghg_s3,2019_ghg_s3,2020_ghg_s3,2021_ghg_s3,2022_ghg_s3,2014_production,2015_production,2016_production,2017_production,2018_production,2019_production,2020_production,2021_production,2022_production
64,WEC Energy Group,549300IGLYTZUK3PVP70,US92939U1060,US,North America,Electricity Utilities,equity,USD,2019,26300000000.0,7523100000.0,38120800000,38158300000.0,34951800000.0,Mt CO2,TWh,21.96727443074169 CO2 * megametric_ton,22.45967009728603 CO2 * megametric_ton,19.020371447730597 CO2 * megametric_ton,19.6272318151656 CO2 * megametric_ton,16.478434173257213 CO2 * megametric_ton,9.874576584949224 CO2 * megametric_ton,9.667297788165046 CO2 * megametric_ton,nan CO2 * megametric_ton,...,22.45967009728603 CO2 * megametric_ton,19.020371447730597 CO2 * megametric_ton,19.6272318151656 CO2 * megametric_ton,16.478434173257213 CO2 * megametric_ton,9.874576584949224 CO2 * megametric_ton,9.667297788165046 CO2 * megametric_ton,nan CO2 * megametric_ton,nan CO2 * megametric_ton,nan CO2 * megametric_ton,nan CO2 * megametric_ton,nan CO2 * megametric_ton,nan CO2 * megametric_ton,nan CO2 * megametric_ton,nan CO2 * megametric_ton,nan CO2 * megametric_ton,24.04241698625209 terawatt_hour,25.975117769196213 terawatt_hour,23.367324105653907 terawatt_hour,23.864635428668937 terawatt_hour,21.139317437999996 terawatt_hour,15.874233993999985 terawatt_hour,15.865285688499995 terawatt_hour,nan terawatt_hour,nan terawatt_hour
38,"NextEra Energy, Inc.",UMI46YPGBLUE4VGNNT48,US65339F1012,US,North America,Electricity Utilities,equity,USD,2019,97905404884.0,19204000000.0,134848404884,135448404884.0,117691000000.0,Mt CO2,TWh,45.7958499323177 CO2 * megametric_ton,46.432626473002635 CO2 * megametric_ton,44.46766953140403 CO2 * megametric_ton,45.15651668584934 CO2 * megametric_ton,45.3626011007061 CO2 * megametric_ton,44.579149470133444 CO2 * megametric_ton,42.25746196611065 CO2 * megametric_ton,nan CO2 * megametric_ton,...,46.432626473002635 CO2 * megametric_ton,44.46766953140403 CO2 * megametric_ton,45.15651668584934 CO2 * megametric_ton,45.3626011007061 CO2 * megametric_ton,44.579149470133444 CO2 * megametric_ton,42.25746196611065 CO2 * megametric_ton,nan CO2 * megametric_ton,nan CO2 * megametric_ton,nan CO2 * megametric_ton,nan CO2 * megametric_ton,nan CO2 * megametric_ton,nan CO2 * megametric_ton,nan CO2 * megametric_ton,nan CO2 * megametric_ton,nan CO2 * megametric_ton,123.47207075395039 terawatt_hour,128.11133619119997 terawatt_hour,127.73700738309996 terawatt_hour,129.14431197200005 terawatt_hour,133.74937911255 terawatt_hour,135.71818765739994 terawatt_hour,137.2781434015 terawatt_hour,nan terawatt_hour,nan terawatt_hour
83,"United Power, Inc.",8945004QAHNC2OHL6926,ZZ00000000088,<NA>,<NA>,Electricity Utilities,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Mt CO2,TWh,nan CO2 * megametric_ton,nan CO2 * megametric_ton,nan CO2 * megametric_ton,nan CO2 * megametric_ton,0.0 CO2 * megametric_ton,0.0 CO2 * megametric_ton,0.0 CO2 * megametric_ton,nan CO2 * megametric_ton,...,nan CO2 * megametric_ton,nan CO2 * megametric_ton,nan CO2 * megametric_ton,0.0 CO2 * megametric_ton,0.0 CO2 * megametric_ton,0.0 CO2 * megametric_ton,nan CO2 * megametric_ton,nan CO2 * megametric_ton,nan CO2 * megametric_ton,nan CO2 * megametric_ton,nan CO2 * megametric_ton,nan CO2 * megametric_ton,nan CO2 * megametric_ton,nan CO2 * megametric_ton,nan CO2 * megametric_ton,nan terawatt_hour,nan terawatt_hour,nan terawatt_hour,nan terawatt_hour,0.0 terawatt_hour,-0.000217 terawatt_hour,-0.000133 terawatt_hour,nan terawatt_hour,nan terawatt_hour
71,Basin Electric Power Coop.,549

In [28]:
with pd.ExcelWriter("../data/processed/template-20220415-output.xlsx", datetime_format="YYYY") as writer:
    template_df.to_excel(writer, sheet_name="ITR input data", index=False)

### Load emissions target data

The RMI power plant data is valid for Scope 1 emissions only.

In [29]:
engine.execute(f"describe {rmi_schema}.emissions_targets").fetchall()

[('parent_name', 'varchar', '', ''),
 ('utility_name', 'varchar', '', ''),
 ('respondent_id', 'integer', '', ''),
 ('year', 'timestamp(6)', '', ''),
 ('target_type', 'varchar', '', ''),
 ('state', 'varchar', '', ''),
 ('co2_historical', 'double', '', ''),
 ('co2_target', 'double', '', ''),
 ('co2_target_all_years', 'double', '', ''),
 ('co2_1point5c', 'double', '', ''),
 ('generation_historical', 'double', '', ''),
 ('generation_projected', 'double', '', ''),
 ('generation_1point5c', 'double', '', ''),
 ('co2_intensity_historical', 'double', '', ''),
 ('co2_intensity_target', 'double', '', ''),
 ('co2_intensity_target_all_years', 'double', '', ''),
 ('co2_intensity_1point5c', 'double', '', '')]

### `targets_df` has all the historical and target emissions data
### `trajectory_df` is derived from historical target emissions data

We also preserve RMI's 1.5 degree target info, which can be presented as a trajectory to compare/contrast corporate targets with RMI's best policy recommendations
* rtg_df is the RMI contribution to targets_df (RMI data frame)
* mtg_df is the Steel contribution to targets_df (MDT data frame)

We do not consider targets/emissions for WIRES ONLY utilities (who have no generation of their own).

We set the LEI information based on our hand-curated GLEIF table, not the LEI info in the RMI and SEC data tables

In [30]:
# Emissions targets are now segregated by states, but we care more about rolling them up to the company level.
# Therefore we sum absolutes (emissions and generation) and re-compute intensities based on the aggregated amounts.

rtg_df = pd.read_sql(f"""
select ET.parent_name as company_name, ET.respondent_id, 'Electricity Utilities' as sector, year(ET.year) as year,
       sum(co2_target) as co2_s1_target,
       sum(co2_historical) as co2_s1_historical,
       sum(co2_target_all_years) as co2_s1_target_all_years,
       sum(co2_1point5C) as co2_s1_1point5C,
       sum(generation_historical) as production_historical,
       sum(generation_projected) as production_projected,
       sum(generation_1point5C) as production_1point5C
from {rmi_schema}.emissions_targets ET
     join (select respondent_id, year
           from {rmi_schema}.operations_emissions_by_tech
           where technology_eia!='Batteries' and technology_eia!='Hydroelectric Pumped Storage'
           group by respondent_id, year) EM
           on ET.respondent_id=EM.respondent_id and ((year(ET.year)>2020 and year(EM.year)=2020) or (ET.year=EM.year) or ((year(ET.year)<2005 and year(EM.year)=2005) ))
     -- join (select parent_name, parent_lei from {rmi_schema}.utility_information group by parent_name, parent_lei) U
     --       on ET.parent_name=U.parent_name
     -- join {dera_schema}.financials_by_lei as F on F.lei=U.parent_lei
where ET.target_type='All'
group by ET.parent_name, ET.respondent_id, year(ET.year)
order by company_name, year
""", engine) # parse_dates=['year']

# We set the LEI information based on our hand-curated GLEIF table, not the LEI info in the RMI and SEC data tables
rtg_df.insert(1, 'company_lei', rtg_df.company_name.str.upper().map(gleif_match))
rtg_df.insert(2, 'company_id', rtg_df.company_lei.map(rmi_lei_dict))
rtg_df.loc[rtg_df.production_historical > 0, 'ei_s1_historical'] = rtg_df.co2_s1_historical / rtg_df.production_historical
rtg_df['production_general'] = rtg_df[['production_historical', 'production_projected']].bfill(axis=1).iloc[:, 0]
rtg_df.loc[rtg_df.production_general > 0, 'ei_s1_target'] = rtg_df.co2_s1_target / rtg_df.production_general
rtg_df.loc[rtg_df.production_general > 0, 'ei_s1_target_all_years'] = rtg_df.co2_s1_target_all_years / rtg_df.production_general
rtg_df.loc[rtg_df.production_1point5C > 0, 'ei_s1_1point5C'] = rtg_df.co2_s1_1point5C / rtg_df.production_1point5C
rtg_df.drop(columns='production_general', inplace=True)
rtg_df = rtg_df[rtg_df.company_lei!='529900L26LIS2V8PWM23'] # American States Water has negative/zero production values that mess things up

In [31]:
for col in rtg_df.columns:
    if col.startswith('co2_'):
        rtg_df[col] = rtg_df[col].astype('pint[Mt CO2]')
    elif col.startswith('production_'):
        rtg_df[col] = rtg_df[col].astype('pint[TWh]')
    elif col.startswith('ei_'):
        rtg_df[col] = rtg_df[col].astype('pint[Mt CO2/TWh]')
rtg_df = rtg_df.convert_dtypes()
print(rtg_df.dtypes)
print(f"len(rtg_df) = {len(rtg_df)}")

company_name                                                   string
company_lei                                                    string
company_id                                                     string
respondent_id                                                   Int64
sector                                                         string
year                                                            Int64
co2_s1_target                              pint[CO2 * megametric_ton]
co2_s1_historical                          pint[CO2 * megametric_ton]
co2_s1_target_all_years                    pint[CO2 * megametric_ton]
co2_s1_1point5C                            pint[CO2 * megametric_ton]
production_historical                             pint[terawatt_hour]
production_projected                              pint[terawatt_hour]
production_1point5C                               pint[terawatt_hour]
ei_s1_historical           pint[CO2 * megametric_ton / terawatt_hour]
ei_s1_target        

### Fix target information comprehensively (mostly fixed with March 2022 update)

1. Where co2_target is set to zero before 2019 and then ramps up to a non-zero number before 2020, clear the target number and replace all target data with historical data
2. Where co2_target is NULL, generation_historical==1, and co2_intensity_historical==0, remove false generation_historical==1 data point.  There is never any generation before generators are operational.
3. Where co2_historical is non-NULL and non-zero, look for outlier data.  If the generation_historical for the outlier data is not an outlier in the generation data, recompute co2_intensity_historical and co2_historical based on non-outlier data
4. Where max(year) < 2020, discard forward-looking projections: they are represented elsewhere
5. Where production_projected is non-NULL and flatline from 2021-2050, replace with OECM production growth values for 'North America' region

In [32]:
print("Step 4: When data is exhausted prior to 2020, discard forward-looking projections represented elsewhere")

step4_df = rtg_df.loc[rtg_df.year==2019, ['respondent_id', 'production_historical']].fillna(0)
step4_index = step4_df[step4_df.production_historical!=0]['respondent_id']
print(f"Initial length of target dataset: {len(rtg_df)}")
print("respondent_id not in index")
print(sorted(rtg_df.loc[~rtg_df.respondent_id.isin(step4_index), 'respondent_id'].drop_duplicates().tolist()))
rtg_df = rtg_df.loc[rtg_df.respondent_id.isin(step4_index)]
print(f"Resulting length of target dataset: {len(rtg_df)}")


Step 4: When data is exhausted prior to 2020, discard forward-looking projections represented elsewhere
Initial length of target dataset: 4706
respondent_id not in index
[9, 11, 20, 21, 24, 25, 27, 30, 31, 54, 63, 71, 77, 96, 117, 126, 127, 156, 168, 175, 196, 416, 428]
Resulting length of target dataset: 4566


The RMI targets only cover S1, so we don't need to compute the non-existent S2 and S3 numbers (until they do provide such).

In [33]:
import warnings

def compute_sums_and_wavg(x):
    zero_Mt_CO2 = Q_(0.0, 'Mt CO2')
    d = { 'co2_s1_by_year':x['co2_s1_target_all_years'].sum(),
          'co2_s2_by_year':zero_Mt_CO2,
          'production_by_year':x[['production_historical', 'production_projected']].bfill(axis=1).iloc[:, 0].sum() }
    return pd.Series(d, index=['co2_s1_by_year', 'co2_s2_by_year', 'production_by_year'])

with warnings.catch_warnings():
    warnings.simplefilter("ignore")

    rmi_targets_df = (rtg_df[rtg_df.year>=2014]
          .groupby(['company_name', 'company_lei', 'company_id', 'sector', 'year']) # grouping automagically sets index
          .apply(compute_sums_and_wavg)
          .sort_values(['company_name', 'year'], ascending=[True, True])
         )
m = (rmi_targets_df.production_by_year!=0)

In [34]:
rmi_targets_df.loc[~m, 'ei_s1_by_year'] = m.map(lambda x: Q_(np.nan, "Mt CO2/TWh"))
rmi_targets_df.loc[~m, 'ei_s2_by_year'] = m.map(lambda x: Q_(np.nan, "Mt CO2/TWh"))
rmi_targets_df.loc[~m, 'ei_s1s2_by_year'] = m.map(lambda x: Q_(np.nan, "Mt CO2/TWh"))
rmi_targets_df.loc[m, 'ei_s1_by_year'] = rmi_targets_df.co2_s1_by_year/rmi_targets_df.production_by_year
rmi_targets_df.loc[m, 'ei_s2_by_year'] = rmi_targets_df.co2_s2_by_year/rmi_targets_df.production_by_year
rmi_targets_df.loc[m, 'ei_s1s2_by_year'] = (rmi_targets_df.co2_s1_by_year+rmi_targets_df.co2_s2_by_year)/rmi_targets_df.production_by_year

In [35]:
rmi_targets_df.loc['Exelon Corp.', :, :, :]

,,,,co2_s1_by_year,co2_s2_by_year,production_by_year,ei_s1_by_year,ei_s2_by_year,ei_s1s2_by_year
company_lei,company_id,sector,year,,,,,,
3SOUA6IRML7435B56G12,US30161N1019,Electricity Utilities,2019,0.0 CO2 * megametric_ton,0.0 CO2 * megametric_ton,0.002087 terawatt_hour,0.0 CO2 * megametric_ton / terawatt_hour,0.0 CO2 * megametric_ton / terawatt_hour,0.0 CO2 * megametric_ton / terawatt_hour


In [36]:
steel_production.iloc[0:2]

/Users/michael/opt/miniconda3/envs/jupyter/lib/python3.10/site-packages/pint_pandas/pint_array.py:648: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  return np.array(qtys, dtype="object", copy=copy)
/Users/michael/opt/miniconda3/envs/jupyter/lib/python3.10/site-packages/pint_pandas/pint_array.py:648: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  return np.array(qtys, dtype="object", copy=copy)


,,,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,...,2041,2042,2043,2044,2045,2046,2047,2048,2049,2050
company_name,company_lei,company_id,,,,,,,,,,,,,,,,,,,,,
AK STEEL HOLDING CORP,529900DT4E7ZNETMVC04,US0015471081,6132700.0,7089200.0,6051800.0,5596200.0,5683400.0,5342200.0,5422332.999999999,5503667.994999998,5586223.014924997,5670016.360148872,...,7412642.793455035,7523832.435356861,7636689.921887212,7751240.27071552,7867508.874776252,7985521.507897895,8105304.330516361,8226883.895474106,8350287.153906216,8475541.461214809
ARCELORMITTAL,2EULGUTUI56JI9SAL165,LU0140205948,85100000.0,84600000.0,83900000.0,85200000.0,83900000.0,84500000.0,69100000.0,62900000.0,63843499.99999999,64801152.499999985,...,84717179.91199829,85987937.61067826,87277756.67483841,88586923.02496098,89915726.87033537,91264462.7733904,92633429.71499124,94022931.1607161,95433275.12812681,96864774.25504872


In [37]:
mdt_production = (steel_production
                  .melt(var_name='year', value_name='production_by_year', ignore_index=False)
                  .dropna()
                  .set_index(['year'], append=True))
# display(mdt_production)
mdt_co2 = pd.concat([steel_co2[scope]
                     .melt(var_name='year', value_name=f"co2_{scope}_by_year", ignore_index=False)
                     .dropna()
                     .set_index(['year'], append=True)
                     for scope in scopes],
                    join='outer', axis=1)
# display(mdt_co2)
mdt_ei = pd.concat([steel_ei[scope]
                    .melt(var_name='year', value_name=f"ei_{scope}_by_year", ignore_index=False)
                    .dropna()
                    .set_index(['year'], append=True)
                    for scope in scopes],
                    join='outer', axis=1)
# display(mdt_ei)

/Users/michael/opt/miniconda3/envs/jupyter/lib/python3.10/site-packages/pint_pandas/pint_array.py:648: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  return np.array(qtys, dtype="object", copy=copy)
/Users/michael/opt/miniconda3/envs/jupyter/lib/python3.10/site-packages/pint_pandas/pint_array.py:648: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  return np.array(qtys, dtype="object", copy=copy)
/Users/michael/opt/miniconda3/envs/jupyter/lib/python3.10/site-packages/pint_pandas/pint_array.py:648: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  return np.array(qtys, dtype="object", copy=copy)
/Users/michael/opt/miniconda3/envs/jupyter/lib/python3.10/site-packages/pint_pandas/pint_array.py:648: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  return np.array(qtys, dtype="object", copy=copy)
/Users/michael/opt/miniconda3/envs/j

In [38]:
steel_targets_df = pd.concat([mdt_production, mdt_co2, mdt_ei], join='outer', axis=1)
steel_targets_df.insert(2, 'sector', 'Steel')
steel_targets_df.set_index(['sector'], append=True, inplace=True)
steel_targets_df = steel_targets_df.reorder_levels(order=['company_name', 'company_lei', 'company_id', 'sector', 'year'])
targets_df = pd.concat([rmi_targets_df, steel_targets_df])[['production_by_year','co2_s1_by_year','co2_s2_by_year','co2_s3_by_year','ei_s1_by_year','ei_s2_by_year','ei_s3_by_year']]

In [39]:
targets_df

production_by_year  \
company_name               company_lei          company_id   sector                year                                     
AES Corp.                  2NUNNB7D43COUIRE5295 US00130H1059 Electricity Utilities 2014         28.63310159 terawatt_hour   
                                                                                   2015        22.738255829 terawatt_hour   
                                                                                   2016        22.186759246 terawatt_hour   
                                                                                   2017  10.959301998338871 terawatt_hour   
                                                                                   2018  13.537873157079572 terawatt_hour   
...                                                                                                                   ...   
TENARIS SA                 549300Y7C05BKC4HZB40 US88031M1099 Steel                 2050          4600926.629014815 Fe_ton   
TERNIUM S.A.               529900QG4KU23TEI2E46 US8808901081 Steel                 2050         16337654.439342635 Fe_ton   
TIMKENSTEEL CORP           549300QZTZWHDE9HJL14 US8873991033 Steel                 2050         2136902.3560039606 Fe_ton   
UNITED STATES STEEL CORP   JNLUVFYJT1OZSIQ24U47 US9129091081 Steel                 2050         23601879.430786483 Fe_ton   
WORTHINGTON INDUSTRIES INC 1WRCIANKYOIK6KYE5E82 US9818111026 Steel                 2050         6263100.7455529915 Fe_ton   

                                                                                                                  co2_s1_by_year  \
company_name               company_lei          company_id   sector                year                                            
AES Corp.                  2NUNNB7D43COUIRE5295 US00130H1059 Electricity Utilities 2014  27.647741534207256 CO2 * megametric_ton   
                                                                                   2015  22.075533730533344 CO2 * megametric_ton   
                                                                                   2016  20.952695203741786 CO2 * megametric_ton   
                                                                                   2017  10.483392150493383 CO2 * megametric_ton   
                                                                                   2018  11.235889892304938 CO2 * megametric_ton   
...                                                                                                                          ...   
TENARIS SA                 549300Y7C05BKC4HZB40 US88031M1099 Steel                 2050                     0.0 CO2 * metric_ton   
TERNIUM S.A.               529900QG4KU23TEI2E46 US8808901081 Steel                 2050                     0.0 CO2 * metric_ton   
TIMKENSTEEL CORP           549300QZTZWHDE9HJL14 US8873991033 Steel                 2050                     0.0 CO2 * metric_ton   
UNITED STATES STEEL CORP   JNLUVFYJT1OZSIQ24U47 US9129091081 Steel                 2050                     0.0 CO2 * metric_ton   
WORTHINGTON INDUSTRIES INC 1WRCIANKYOIK6KYE5E82 US9818111026 Steel                 2050                     0.0 CO2 * metric_ton   

                                                                                                   co2_s2_by_year  \
company_name               company_lei          company_id   sector                year                             
AES Corp.                  2NUNNB7D43COUIRE5295 US00130H1059 Electricity Utilities 2014  0.0 CO2 * megametric_ton   
                                                                                   2015  0.0 CO2 * megametric_ton   
                                                                                   2016  0.0 CO2 * megametric_ton   
                                                                                   2017  0.0 CO2 * megametric_ton   
                                                          

In [40]:
targets_df.loc['WORTHINGTON INDUSTRIES INC']

production_by_year  \
company_lei          company_id   sector year                              
1WRCIANKYOIK6KYE5E82 US9818111026 Steel  2014           3282000.0 Fe_ton   
                                         2015           3510000.0 Fe_ton   
                                         2016           3523000.0 Fe_ton   
                                         2017           4070000.0 Fe_ton   
                                         2018           3820000.0 Fe_ton   
                                         2019           3715000.0 Fe_ton   
                                         2020           3830000.0 Fe_ton   
                                         2021           4067000.0 Fe_ton   
                                         2022  4128004.9999999995 Fe_ton   
                                         2023   4189925.074999999 Fe_ton   
                                         2024   4252773.951124999 Fe_ton   
                                         2025   4316565.560391873 Fe_ton   
                                         2026    4381314.04379775 Fe_ton   
                                         2027   4447033.754454715 Fe_ton   
                                         2028   4513739.260771535 Fe_ton   
                                         2029   4581445.349683109 Fe_ton   
                                         2030   4650167.029928355 Fe_ton   
                                         2031   4719919.535377279 Fe_ton   
                                         2032   4790718.328407938 Fe_ton   
                                         2033   4862579.103334056 Fe_ton   
                                         2034   4935517.789884066 Fe_ton   
                                         2035   5009550.556732326 Fe_ton   
                                         2036    5084693.81508331 Fe_ton   
                                         2037    5160964.22230956 Fe_ton   
                                         2038   5238378.685644202 Fe_ton   
                                         2039   5316954.365928864 Fe_ton   
                                         2040   5396708.681417797 Fe_ton   
                                         2041   5477659.311639062 Fe_ton   
                                         2042   5559824.201313647 Fe_ton   
                                         2043   5643221.564333351 Fe_ton   
                                         2044   5727869.887798351 Fe_ton   
                                         2045   5813787.936115325 Fe_ton   
                                         2046   5900994.755157055 Fe_ton   
                                         2047    5989509.67648441 Fe_ton   
                                         2048   6079352.321631676 Fe_ton   
                                         2049   6170542.606456149 Fe_ton   
                                         2050  6263100.7455529915 Fe_ton   

                                                                    co2_s1_by_year  \
company_lei          company_id   sector year                                        
1WRCIANKYOIK6KYE5E82 US9818111026 Steel  2014            131853.0 CO2 * metric_ton   
                                         2015            131853.0 CO2 * metric_ton   
                                         2016            126399.0 CO2 * metric_ton   
                                         2017            132944.0 CO2 * metric_ton   
                                         2018            139953.0 CO2 * metric_ton   
                                         2019            134257.0 CO2 * metric_ton   
                                         2020            130506.0 CO2 * metric_ton   
                                         2021           130051.17 CO2 * metric_ton   
                                         2022           129596.34 CO2 * metric_ton   
                                         2023           129141.51 CO2 * metric_ton   
                                         2024           1

In [41]:
targets_df.unstack(level='year')['ei_s1_by_year'].sample(15).sort_index(level=['company_name'], ascending=[1])

,,,year,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,...,2041,2042,2043,2044,2045,2046,2047,2048,2049,2050
company_name,company_lei,company_id,sector,,,,,,,,,,,,,,,,,,,,,
AK STEEL HOLDING CORP,529900DT4E7ZNETMVC04,US0015471081,Steel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Alliant Energy,5493009ML300G373MZ12,US0188021085,Electricity Utilities,0.8442260959160253 CO2 * megametric_ton / tera...,0.7875231657500559 CO2 * megametric_ton / tera...,0.7433087004936753 CO2 * megametric_ton / tera...,0.7324798682922219 CO2 * megametric_ton / tera...,0.6729058287604113 CO2 * megametric_ton / tera...,0.5407611872794243 CO2 * megametric_ton / tera...,0.5015295973237168 CO2 * megametric_ton / tera...,0.48748168715774215 CO2 * megametric_ton / ter...,0.4734337769917675 CO2 * megametric_ton / tera...,0.4593858668257927 CO2 * megametric_ton / tera...,...,0.16388451383676697 CO2 * megametric_ton / ter...,0.1458319890535685 CO2 * megametric_ton / tera...,0.12777946427037004 CO2 * megametric_ton / ter...,0.10972693948717155 CO2 * megametric_ton / ter...,0.09167441470397306 CO2 * megametric_ton / ter...,0.07362188992077459 CO2 * megametric_ton / ter...,0.05556936513757612 CO2 * megametric_ton / ter...,0.03751684035437764 CO2 * megametric_ton / ter...,0.019464315571179162 CO2 * megametric_ton / te...,0.0014117907879806877 CO2 * megametric_ton / t...
"American Electric Power Co., Inc.",1B4S6S7G0TW5EE83BO58,US0255371017,Electricity Utilities,0.7696003652349954 CO2 * megametric_ton / tera...,0.7382718525604623 CO2 * megametric_ton / tera...,0.7105421907842547 CO2 * megametric_ton / tera...,0.713816351559896 CO2 * megametric_ton / teraw...,0.710282260190262 CO2 * megametric_ton / teraw...,0.692327450150574 CO2 * megametric_ton / teraw...,0.6128452255619268 CO2 * megametric_ton / tera...,0.5758021342418952 CO2 * megametric_ton / tera...,0.5386281397668047 CO2 * megametric_ton / tera...,0.5013241927747065 CO2 * megametric_ton / tera...,...,0.10824811723535402 CO2 * megametric_ton / ter...,0.09635751030378786 CO2 * megametric_ton / ter...,0.08443187476608145 CO2 * megametric_ton / ter...,0.07247154512090435 CO2 * megametric_ton / ter...,0.06047685809210219 CO2 * megametric_ton / ter...,0.048448152602757304 CO2 * megametric_ton / te...,0.036385769748982305 CO2 * megametric_ton / te...,0.024290052773451015 CO2 * megametric_ton / te...,0.012161347038670669 CO2 * megametric_ton / te...,0.0 CO2 * megametric_ton / terawatt_hour
"Berkshire Hathaway, Inc.",5493000C01ZX7D35SD85,US0846707026,Electricity Utilities,0.6909976160710452 CO2 * megametric_ton / tera...,0.6634691095179501 CO2 * megametric_ton / tera...,0.608936214328231 CO2 * megametric_ton / teraw...,0.6137741583276948 CO2 * megametric_ton / tera...,0.6142798212224332 CO2 * megametric_ton / tera...,0.5814533676984677 CO2 * megametric_ton / tera...,0.5159816702730536 CO2 * megametric_ton / tera...,0.4846120985400192 CO2 * megametric_ton / tera...,0.4535609930913395 CO2 * megametric_ton / tera...,0.42282711644942367 CO2 * megametric_ton / ter...,...,0.09609028509725176 CO2 * megametric_ton / ter...,0.08583981263446382 CO2 * megametric_ton / ter...,0.07570247338908442 CO2 * megametric_ton / ter...,0.06567771772400498 CO2 * megametric_ton / ter...,0.05576499252035092 CO2 * megametric_ton / ter...,0.04596374126189991 CO2 * megametric_ton / ter...,0.0362734041194384 CO2 * megametric_ton / tera...,0.02669341803503675 CO2 * megametric_ton / ter...,0.0172232168062233 CO2 * megametric_ton / tera...,0.007862231170037811 CO2 * megametric_ton / te...
Dominion Energy,ILUL7B6Z54MRYCF6H308,US25746U1097,Electricity Utilities,0.47509713923545777 CO2 * megametric_ton / ter...,0.4495306199094849 CO2 * megametric_ton / tera...,0.4233977689508571 CO2 * megametric_ton / tera...,0.38661977596226405 CO2 * megametric_ton / ter...,0.3668902273760904 CO2 * megametric_ton / tera...,0.3215032375471965 CO2 * megametric_ton / tera...,0.31897578283416933 CO2 * megametric_ton / ter...,0.30389

In [42]:
traj_df = {}
traj_mdf = {}
traj_udf = targets_df.unstack(level='year')
for scope in scopes:
    # We start by copying the target data, but we will use only the historic and replace the projection
    traj_df[scope] = traj_udf[f"ei_{scope}_by_year"].copy()
    # By calculating 2014-2019, we miss the anomoly of 2020
    historic_progress = (traj_df[scope][2019] / traj_df[scope][2014]).dropna().map(lambda x: x.m)

    # There are wierd artifacts where energy storage systems have negative generation, so treat their progress as zero
    # If intensity is actually growing, cap trajectory at 1 (no progress).
    annualized_progress = historic_progress.where(historic_progress>=0, 0).where(historic_progress<=1, 1) ** (1/(2019-2014))

    for year in range(2020,2051):
        traj_df[scope].loc[:, year] = traj_df[scope][2020] * annualized_progress ** (year-2020)
    traj_mdf[scope] = (traj_df[scope]
                       .melt(var_name='year', value_name=f"ei_{scope}_by_year", ignore_index=False)
                       .set_index('year',append=True)
                       .convert_dtypes())

traj_mdf = pd.concat([*traj_mdf.values()], join='outer', axis=1)
traj_mdf.loc[targets_df.index.intersection(traj_mdf.index), 'production_by_year'] = targets_df['production_by_year']
display(traj_mdf.loc['CLEVELAND-CLIFFS INC'])

ei_s1_by_year  \
company_lei          company_id   sector year                                                  
549300TM2WLI2BJMDD86 US1858991011 Steel  2014    0.384718670726176 CO2 * metric_ton / Fe_ton   
                                         2015  0.35742364177896635 CO2 * metric_ton / Fe_ton   
                                         2016  0.36919177409457066 CO2 * metric_ton / Fe_ton   
                                         2017  0.35637795124245286 CO2 * metric_ton / Fe_ton   
                                         2018  0.34643674680532327 CO2 * metric_ton / Fe_ton   
                                         2019  0.33781346015346797 CO2 * metric_ton / Fe_ton   
                                         2020   0.3436249356218983 CO2 * metric_ton / Fe_ton   
                                         2021   0.3348045942178267 CO2 * metric_ton / Fe_ton   
                                         2022  0.32621065786885856 CO2 * metric_ton / Fe_ton   
                                         2023   0.3178373150937111 CO2 * metric_ton / Fe_ton   
                                         2024  0.30967890358318934 CO2 * metric_ton / Fe_ton   
                                         2025   0.3017299063711598 CO2 * metric_ton / Fe_ton   
                                         2026   0.2939849481038106 CO2 * metric_ton / Fe_ton   
                                         2027   0.2864387914046731 CO2 * metric_ton / Fe_ton   
                                         2028  0.27908633333294913 CO2 * metric_ton / Fe_ton   
                                         2029   0.2719226019327467 CO2 * metric_ton / Fe_ton   
                                         2030  0.26494275287089236 CO2 * metric_ton / Fe_ton   
                                         2031  0.25814206616104546 CO2 * metric_ton / Fe_ton   
                                         2032  0.25151594297190005 CO2 * metric_ton / Fe_ton   
                                         2033  0.24505990251731502 CO2 * metric_ton / Fe_ton   
                                         2034  0.23876957902627013 CO2 * metric_ton / Fe_ton   
                                         2035  0.23264071879059886 CO2 * metric_ton / Fe_ton   
                                         2036  0.22666917728850156 CO2 * metric_ton / Fe_ton   
                                         2037  0.22085091638189347 CO2 * metric_ton / Fe_ton   
                                         2038  0.21518200158569314 CO2 * metric_ton / Fe_ton   
                                         2039  0.20965859940720372 CO2 * metric_ton / Fe_ton   
                                         2040  0.20427697475378853 CO2 * metric_ton / Fe_ton   
                                         2041  0.19903348840708784 CO2 * metric_ton / Fe_ton   
                                         2042  0.19392459456206868 CO2 * metric_ton / Fe_ton   
                                         2043  0.18894683842924334 CO2 * metric_ton / Fe_ton   
                                         2044  0.18409685389843602 CO2 * metric_ton / Fe_ton   
                                         2045   0.1793713612625162 CO2 * metric_ton / Fe_ton   
                                         2046  0.17476716499956138 CO2 * metric_ton / Fe_ton   
                                         2047  0.17028115161194735 CO2 * metric_ton / Fe_ton   
                                         2048  0.16591028752090686 CO2 * metric_ton / Fe_ton   
                                         2049  0.16165161701513112 CO2 * metric_ton / Fe_ton   
                                         2050  0.15750226025202776 CO2 * metric_ton / Fe_ton   

                                                                                ei_s2_by_year  \
company_lei          company_id   sector year                                                   
549300TM2WLI2BJMDD86 US1858991011 Steel  2014  0.049265325603802555 CO2 * metric_ton / Fe_ton   
                                         20

In [43]:
df = traj_mdf[['ei_s1_by_year','ei_s2_by_year','ei_s3_by_year']].multiply(traj_mdf['production_by_year'], axis='index')
df.rename(columns={f"ei_{scope}_by_year":f"co2_{scope}_by_year" for scope in scopes}, inplace=True)
trajectories_df = pd.concat([df, traj_mdf], axis=1)
trajectories_df = trajectories_df[[trajectories_df.columns[-1]] + list(trajectories_df.columns[0:-1])]
trajectories_df

,,,,,production_by_year,co2_s1_by_year,co2_s2_by_year,co2_s3_by_year,ei_s1_by_year,ei_s2_by_year,ei_s3_by_year
company_name,company_lei,company_id,sector,year,,,,,,,
AES Corp.,2NUNNB7D43COUIRE5295,US00130H1059,Electricity Utilities,2014,28.63310159 terawatt_hour,27.647741534207256 CO2 * megametric_ton,0.0 CO2 * megametric_ton,nan terawatt_hour,0.9655866811111767 CO2 * megametric_ton / tera...,0.0 CO2 * megametric_ton / terawatt_hour,NaN
AK STEEL HOLDING CORP,529900DT4E7ZNETMVC04,US0015471081,Steel,2014,6132700.0 Fe_ton,nan Fe_ton,nan Fe_ton,nan Fe_ton,NaN,NaN,NaN
"ALLETE, Inc.",549300NNLSIMY6Z8OT86,US0185223007,Electricity Utilities,2014,10.179607599645534 terawatt_hour,9.31812469169611 CO2 * megametric_ton,0.0 CO2 * megametric_ton,nan terawatt_hour,0.9153716978266017 CO2 * megametric_ton / tera...,0.0 CO2 * megametric_ton / terawatt_hour,NaN
ARCELORMITTAL,2EULGUTUI56JI9SAL165,LU0140205948,Steel,2014,85100000.0 Fe_ton,nan Fe_ton,nan Fe_ton,nan Fe_ton,NaN,NaN,NaN
Algonquin Power & Utilities Corp.,549300K5VIUTJXQL7X75,US0158577090,Electricity Utilities,2014,0.0001995 terawatt_hour,0.0001890439769084 CO2 * megametric_ton,0.0 CO2 * megametric_ton,nan terawatt_hour,0.9475888566837093 CO2 * megametric_ton / tera...,0.0 CO2 * megametric_ton / terawatt_hour,NaN
...,...,...,...,...,...,...,...,...,...,...,...
UNITED STATES STEEL CORP,JNLUVFYJT1OZSIQ24U47,US9129091081,Steel,2050,23601879.430786483 Fe_ton,50107976.2612018 CO2 * metric_ton,4090447.0417307597 CO2 * metric_ton,nan Fe_ton,2.123050259965338 CO2 * metric_ton / Fe_ton,0.1733102253032929 CO2 * metric_ton / Fe_ton,NaN
Valtec Power,RMI00000000000000015,ZZ00000000015,Electricity Utilities,2050,0.012981 terawatt_hour,0.0 CO2 * megametric_ton,0.0 CO2 * megametric_ton,nan terawatt_hour,0.0 CO2 * megametric_ton / terawatt_hour,0.0 CO2 * megametric_ton / terawatt_hour,NaN
WEC Energy Group,549300IGLYTZUK3PVP70,US92939U1060,Electricity Utilities,2050,15.865285688499998 terawatt_hour,0.9104202357338959 CO2 * megametric_ton,0.0 CO2 * megametric_ton,nan terawatt_hour,0.05738442115756017 CO2 * megametric_ton / ter...,0.0 CO2 * megametric_ton / terawatt_hour,NaN


In [44]:
targets_df.sort_index(level=['company_name','company_lei','company_id','sector','year'], ascending=[1, 1, 1, 1, 1], inplace=True)
trajectories_df.sort_index(level=['company_name','company_lei','company_id','sector','year'], ascending=[1, 1, 1, 1, 1], inplace=True)

In [45]:
targets_df.loc[(slice(None), slice(None), slice(None), slice(None), slice(2019,2024))]

production_by_year  \
company_name      company_lei          company_id   sector                year                                     
AES Corp.         2NUNNB7D43COUIRE5295 US00130H1059 Electricity Utilities 2019        15.292476505 terawatt_hour   
                                                                          2020         13.07516751 terawatt_hour   
                                                                          2021  13.166747153195725 terawatt_hour   
                                                                          2022  13.258977045397394 terawatt_hour   
                                                                          2023    13.3518618036111 terawatt_hour   
...                                                                                                          ...   
Xcel Energy, Inc. LGJNMI9GH8XIDG5RCM61 US98389B1008 Electricity Utilities 2020   69.48078152007834 terawatt_hour   
                                                                          2021   69.96114096740432 terawatt_hour   
                                                                          2022   70.44599284966671 terawatt_hour   
                                                                          2023   70.93538774699746 terawatt_hour   
                                                                          2024   71.42937685734277 terawatt_hour   

                                                                                                         co2_s1_by_year  \
company_name      company_lei          company_id   sector                year                                            
AES Corp.         2NUNNB7D43COUIRE5295 US00130H1059 Electricity Utilities 2019  11.616368146942127 CO2 * megametric_ton   
                                                                          2020   9.425520125987731 CO2 * megametric_ton   
                                                                          2021    8.90997288882708 CO2 * megametric_ton   
                                                                          2022   8.386820120893065 CO2 * megametric_ton   
                                                                          2023   7.855979164131556 CO2 * megametric_ton   
...                                                                                                                 ...   
Xcel Energy, Inc. LGJNMI9GH8XIDG5RCM61 US98389B1008 Electricity Utilities 2020  34.879480047385336 CO2 * megametric_ton   
                                                                          2021   32.60620611821284 CO2 * megametric_ton   
                                                                          2022  30.332932189040342 CO2 * megametric_ton   
                                                                          2023  28.059658259867838 CO2 * megametric_ton   
                                                                          2024   25.78638433069534 CO2 * megametric_ton   

                                                                                          co2_s2_by_year  \
company_name      company_lei          company_id   sector                year                             
AES Corp.         2NUNNB7D43COUIRE5295 US00130H1059 Electricity Utilities 2019  0.0 CO2 * megametric_ton   
                                                                          2020  0.0 CO2 * megametric_ton   
                                                                          2021  0.0 CO2 * megametric_ton   
                                                                          2022  0.0 CO2 * megametric_ton   
                                                                          2023  0.0 CO2 * megametric_ton   
...                                                                                                  ...   
Xcel Energy, Inc. LGJNMI9GH8XIDG5RCM61 US98389B1008 Electricity Utilities 2020  0.0 CO2 * megametric_ton   
             

In [46]:
trajectories_df.loc[(slice(None), slice(None), slice(None), slice(None), slice(2019,2024))]

production_by_year  \
company_name      company_lei          company_id   sector                year                                     
AES Corp.         2NUNNB7D43COUIRE5295 US00130H1059 Electricity Utilities 2019        15.292476505 terawatt_hour   
                                                                          2020         13.07516751 terawatt_hour   
                                                                          2021  13.166747153195725 terawatt_hour   
                                                                          2022  13.258977045397394 terawatt_hour   
                                                                          2023    13.3518618036111 terawatt_hour   
...                                                                                                          ...   
Xcel Energy, Inc. LGJNMI9GH8XIDG5RCM61 US98389B1008 Electricity Utilities 2020   69.48078152007834 terawatt_hour   
                                                                          2021   69.96114096740432 terawatt_hour   
                                                                          2022   70.44599284966671 terawatt_hour   
                                                                          2023   70.93538774699746 terawatt_hour   
                                                                          2024   71.42937685734277 terawatt_hour   

                                                                                                         co2_s1_by_year  \
company_name      company_lei          company_id   sector                year                                            
AES Corp.         2NUNNB7D43COUIRE5295 US00130H1059 Electricity Utilities 2019  11.616368146942127 CO2 * megametric_ton   
                                                                          2020   9.425520125987731 CO2 * megametric_ton   
                                                                          2021   9.046838031618789 CO2 * megametric_ton   
                                                                          2022   8.683375739702083 CO2 * megametric_ton   
                                                                          2023   8.334521273584494 CO2 * megametric_ton   
...                                                                                                                 ...   
Xcel Energy, Inc. LGJNMI9GH8XIDG5RCM61 US98389B1008 Electricity Utilities 2020  34.879480047385336 CO2 * megametric_ton   
                                                                          2021   33.44290853368327 CO2 * megametric_ton   
                                                                          2022   32.06603791828952 CO2 * megametric_ton   
                                                                          2023  30.746366833942968 CO2 * megametric_ton   
                                                                          2024  29.481499484978226 CO2 * megametric_ton   

                                                                                          co2_s2_by_year  \
company_name      company_lei          company_id   sector                year                             
AES Corp.         2NUNNB7D43COUIRE5295 US00130H1059 Electricity Utilities 2019  0.0 CO2 * megametric_ton   
                                                                          2020  0.0 CO2 * megametric_ton   
                                                                          2021  0.0 CO2 * megametric_ton   
                                                                          2022  0.0 CO2 * megametric_ton   
                                                                          2023  0.0 CO2 * megametric_ton   
...                                                                                                  ...   
Xcel Energy, Inc. LGJNMI9GH8XIDG5RCM61 US98389B1008 Electricity Utilities 2020  0.0 CO2 * megametric_ton   
             

### TODO: Implement Units

Intensity and Production data need Units to distinguish TWh of generation vs. Tons of Steel production

Company data is converted to USD by SEC_DERA ingestion for now, but should support any currencies in the future

In [47]:
# If DF_COL contains Pint quantities (because it is a PintArray or an array of Pint Quantities),
# return a two-column dataframe of magnitudes and units.
# If DF_COL contains no Pint quanities, return it unchanged.

def dequantify_column(df_col: pd.Series) -> pd.DataFrame:
    if type(df_col.values)==PintArray:
        return pd.DataFrame({df_col.name: df_col.values.quantity.m,
                             df_col.name + "_units": str(df_col.values.dtype.units)},
                            index=df_col.index)
    elif df_col.size==0:
        return df_col
    elif df_col.map(lambda x: isinstance(x, Quantity)).any():
        values = df_col.map(lambda x: (x.m, x.u) if isinstance(x, Quantity) else x)
        return pd.DataFrame({df_col.name: df_col.map(lambda x: x.m if isinstance(x, Quantity) else x),
                             df_col.name + "_units": df_col.map(lambda x: str(x.u) if isinstance(x, Quantity) else None)},
                            index=df_col.index)
    else:
        return df_col

# Rewrite dataframe DF so that columns containing Pint quantities are represented by a column for the Magnitude and column for the Units.
# The magnitude column retains the original column name and the units column is renamed with a _units suffix.
def dequantify_df(df: pd.DataFrame) -> pd.DataFrame:
    return pd.concat([dequantify_column(df[col]) for col in df.columns], axis=1)

In [48]:
# Because this DF comes from reading a Trino table, and because columns must be unqiue, we don't have to enumerate to ensure we properly handle columns with duplicated names

def requantify_df(df: pd.DataFrame) -> pd.DataFrame:
    units_col = None
    columns_reversed = reversed(df.columns)
    for col in columns_reversed:
        if col.endswith("_units"):
            if units_col:
                # We expect _units column to follow a non-units column
                raise ValueError
            units_col = col
            continue
        if units_col:
            if col + '_units' != units_col:
                raise ValueError
            if (df[units_col]==df[units_col][0]).all():
                # Make a PintArray
                new_col = PintArray(df[col], dtype=f"pint[{ureg(df[units_col][0]).u}]")
            else:
                # Make a pd.Series of Quantity in a way that does not throw UnitStrippedWarning
                new_col = pd.Series(data=df[col], name=col) * pd.Series(data=df[units_col].map(lambda x: ureg(x).u), name=col)
            df = df.drop(columns=units_col)
            df[col] = new_col
            units_col = None
    return df

In [49]:
tablenames = 'company_data', 'target_data', 'trajectory_data', 'emissions_data', 'production_data'

In [50]:
schema_create = engine.execute(f"""
CREATE SCHEMA if not exists {ingest_catalog}.{demo_schema}
 AUTHORIZATION USER michaeltiemannosc
 WITH (
     location = 's3a://osc-datacommons-s3-bucket-dev02/data/demo_dv.db'
 )
""")
schema_create.fetchall()

[(True,)]

In [51]:
targets_to_sql = dequantify_df(targets_df.drop(columns='production_by_year'))
targets_to_sql.loc[:, :, :, 'Steel']

co2_s1_by_year  \
company_name               company_lei          company_id   year                   
AK STEEL HOLDING CORP      529900DT4E7ZNETMVC04 US0015471081 2014             NaN   
                                                             2015             NaN   
                                                             2016             NaN   
                                                             2017             NaN   
                                                             2018             NaN   
...                                                                           ...   
WORTHINGTON INDUSTRIES INC 1WRCIANKYOIK6KYE5E82 US9818111026 2046       25191.540   
                                                             2047       18893.655   
                                                             2048       12595.770   
                                                             2049        6297.885   
                                                             2050           0.000   

                                                                  co2_s1_by_year_units  \
company_name               company_lei          company_id   year                        
AK STEEL HOLDING CORP      529900DT4E7ZNETMVC04 US0015471081 2014                 None   
                                                             2015                 None   
                                                             2016                 None   
                                                             2017                 None   
                                                             2018                 None   
...                                                                                ...   
WORTHINGTON INDUSTRIES INC 1WRCIANKYOIK6KYE5E82 US9818111026 2046     CO2 * metric_ton   
                                                             2047     CO2 * metric_ton   
                                                             2048     CO2 * metric_ton   
                                                             2049     CO2 * metric_ton   
                                                             2050     CO2 * metric_ton   

                                                                   co2_s2_by_year  \
company_name               company_lei          company_id   year                   
AK STEEL HOLDING CORP      529900DT4E7ZNETMVC04 US0015471081 2014             NaN   
                                                             2015             NaN   
                                                             2016             NaN   
                                                             2017             NaN   
                                                             2018             NaN   
...                                                                           ...   
WORTHINGTON INDUSTRIES INC 1WRCIANKYOIK6KYE5E82 US9818111026 2046        24634.12   
                                                             2047        18475.59   
                                                             2048        12317.06   
                                                             2049         6158.53   
                                                             2050            0.00   

                                                                  co2_s2_by_year_units  \
company_name               company_lei          company_id   year                        
AK STEEL HOLDING CORP      529900DT4E7ZNETMVC04 US0015471081 2014                 None   
                                                             2015                 None   
                                                             2016                 None   
                                                             2017                 None   
                                                             2018                 None   
...                                                   

In [52]:
dataframes = [financial_df.loc[financial_df.index.intersection(targets_df.reset_index('year').index)].reset_index().convert_dtypes(),
              dequantify_df(targets_df.drop(columns='production_by_year')).reset_index().convert_dtypes(),
              dequantify_df(trajectories_df.drop(columns='production_by_year')).reset_index().convert_dtypes(),
              dequantify_df(targets_df[['co2_s1_by_year', 'co2_s2_by_year']]).reset_index().convert_dtypes(),
              dequantify_df(targets_df[['production_by_year']]).reset_index().convert_dtypes(),]

for ingest_table, df in zip(tablenames, dataframes):
    drop_table = engine.execute(f"drop table if exists {demo_schema}.{ingest_table}")
    drop_table.fetchall()

    columnschema = osc.create_table_schema_pairs(df)

    tabledef = f"""
create table if not exists {ingest_catalog}.{demo_schema}.{ingest_table}(
{columnschema}
) with (
    format = 'ORC',
    partitioning = array['year']
)
"""

    print(tabledef)
    qres = engine.execute(tabledef)
    print(qres.fetchall())
    df.to_sql(ingest_table,
              con=engine, schema=demo_schema, if_exists='append',
              index=False,
              method=osc.TrinoBatchInsert(batch_size = 2000, verbose = True))


create table if not exists osc_datacommons_dev.demo_dv.company_data(
    company_name varchar,
    company_lei varchar,
    company_id varchar,
    sector varchar,
    country varchar,
    region varchar,
    exposure varchar,
    currency varchar,
    year bigint,
    company_market_cap double,
    company_revenue double,
    company_ev bigint,
    company_evic double,
    company_total_assets double,
    company_cash_equivalents double,
    company_debt double
) with (
    format = 'ORC',
    partitioning = array['year']
)

[(True,)]
constructed fully qualified table name as: "demo_dv.company_data"
inserting 55 records
  ('AES Corp.', '2NUNNB7D43COUIRE5295', 'US00130H1059', 'Electricity Utilities', 'US', 'North America', 'equity', 'USD', 2019, 10870000000.0, 10189000000.0, 10102000000, 11131000000.0, 33648000000.0, 1029000000.0, 261000000.0)
  ('ALLETE, Inc.', '549300NNLSIMY6Z8OT86', 'US0185223007', 'Electricity Utilities', 'US', 'North America', 'equity', 'USD', 2019, 4285299935.0,

In [53]:
pdf = targets_df.pivot(index=['company_name', 'company_lei', 'company_id'], columns='year').reset_index()

KeyError: "None of ['company_name', 'company_lei', 'company_id', 'year'] are in the columns"

In [ ]:
pdf

In [ ]:
stop!
# pdf.insert(1, 'company_lei', pdf.company_name.str.upper().map(gleif_match))
# pdf.insert(2, 'company_id', pdf.company_lei.map(rmi_lei_dict))
# pdf = pdf.set_index(['company_name','company_lei', 'company_id'], drop=True)
pdf.columns.names=[None,None]
pdf

In [ ]:
ei_s1_df = pd.concat([pdf.company_name, pdf.company_lei, pdf.company_id, pdf.ei_s1_target_by_year.reset_index()], axis=1).drop('index', axis=1)
ei_s1_df

In [ ]:
ei_s2_df = pd.concat([pdf.company_name, pdf.company_lei, pdf.company_id, pdf.ei_s2_target_by_year.reset_index()], axis=1).drop('index', axis=1)
ei_s2_df

In [ ]:
ei_s1_df.iloc[:, 3] = 2*ei_s1_df.iloc[:, 4] - ei_s1_df.iloc[:, 5]
ei_s1_df = ei_s1_df[ei_s1_df.company_id.notna()]
ei_s1_df.insert(3, 'scope', 'S1')
ei_s1_df.head(10)

In [ ]:
ei_s2_df.iloc[:, 3] = 2*ei_s2_df.iloc[:, 4] - ei_s2_df.iloc[:, 5]
ei_s2_df = ei_s2_df[ei_s2_df.company_id.notna()]
ei_s2_df.insert(3, 'scope', 'S2')
ei_s2_df.head(10)

In [ ]:
ei_s1_df.iloc[:, 3] = 2*ei_s1_df.iloc[:, 4] - ei_s1_df.iloc[:, 5]
ei_s1_df = co2_ei_df[co2_ei_df.company_id.notna()]
ei_s1_df.insert(3, 'scope', 'S1')
ei_s1_df.head(10)

In [ ]:
co2_df = pd.concat([pdf.company_name, pdf.company_lei, pdf.company_id, pdf.co2_target_by_year.reset_index()], axis=1).drop('index', axis=1)
co2_df = co2_df[co2_df.company_id.notna()]
co2_df.insert(3, 'scope', 'S1+S2')
co2_df.head()

In [ ]:
gen_df = pd.concat([pdf.company_name, pdf.company_lei, pdf.company_id, pdf.production_by_year.reset_index()], axis=1).drop('index', axis=1)
gen_df.iloc[:, 3] = 2*gen_df.iloc[:, 4] - gen_df.iloc[:, 5]
gen_df = gen_df[gen_df.company_id.notna()]
gen_df.insert(3, 'production', 'TWh')
gen_df.head()

In [ ]:
with pd.ExcelWriter("rmi-20220307-output.xlsx", datetime_format="YYYY") as writer:
    financial_df.to_excel(writer, sheet_name="fundamental_data", index=False)
    co2_ei_df.to_excel(writer, sheet_name="projected_ei_in_Wh", index=False)
    gen_df.to_excel(writer, sheet_name="projected_production", index=False)
    co2_df.to_excel(writer, sheet_name="projected_co2", index=False)


In [ ]:
portfolio_zero = portfolio_df.copy()
portfolio_zero.target_probability = 0.0
portfolio_one = portfolio_df.copy()
portfolio_one.target_probability = 1.0

portfolio_df.to_csv("rmi-20220307-portfolio.csv", sep=';', index=False)

In [ ]:
engine.execute(f"select count (*) from (select parent_name from {rmi_schema}.utility_information group by parent_name)").fetchall()

If the following is non-NULL, the Data Vault will reject the company data

In [ ]:
engine.execute(f"select C.company_name, C.company_id, EI.* from {demo_schema}.company_data C left join {demo_schema}.intensity_data EI on EI.company_name=C.company_name where EI.co2_intensity_target_by_year is NULL").fetchall()